In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.preprocessing import Imputer
from time import time
#import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

#from fancyimpute import KNN

In [4]:
#traindf = pd.read_csv('/Users/Vishy/Files/AVDatahack/Churn/RawData/train.csv')
traindf = pd.read_csv('/data/train.csv')
print (traindf.shape)
traindf.head()

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (294,299,300,302,307,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(300000, 377)


,UCIC_ID,NO_OF_Accs,HNW_CATEGORY,vintage,EMAIL_UNSUBSCRIBE,OCCUP_ALL_NEW,city,dependents,zip,FINAL_WORTH_prev1,ENGAGEMENT_TAG_prev1,C_prev1,D_prev1,ATM_C_prev1,ATM_D_prev1,BRANCH_C_prev1,BRANCH_D_prev1,IB_C_prev1,IB_D_prev1,MB_C_prev1,MB_D_prev1,POS_C_prev1,POS_D_prev1,count_C_prev1,count_D_prev1,COUNT_ATM_C_prev1,COUNT_ATM_D_prev1,COUNT_BRANCH_C_prev1,COUNT_BRANCH_D_prev1,COUNT_IB_C_prev1,COUNT_IB_D_prev1,COUNT_MB_C_prev1,COUNT_MB_D_prev1,COUNT_POS_C_prev1,COUNT_POS_D_prev1,custinit_CR_amt_prev1,custinit_DR_amt_prev1,custinit_CR_cnt_prev1,custinit_DR_cnt_prev1,ATM_amt_prev1,ATM_CW_Amt_prev1,ATM_CW_Cnt_prev1,BRN_CW_Amt_prev1,BRN_CW_Cnt_prev1,BRN_CASH_Dep_Amt_prev1,BRN_CASH_Dep_Cnt_prev1,CNR_prev1,BAL_prev1,EOP_prev1,CR_AMB_Prev1,C_prev2,D_prev2,ATM_C_prev2,ATM_D_prev2,BRANCH_C_prev2,BRANCH_D_prev2,IB_C_prev2,IB_D_prev2,MB_C_prev2,MB_D_prev2,POS_C_prev2,POS_D_prev2,count_C_prev2,count_D_prev2,COUNT_ATM_C_prev2,COUNT_ATM_D_prev2,COUNT_BRANCH_C_prev2,COUNT_BRANCH_D_prev2,COUNT_IB_C_prev2,COUNT_IB_D_prev2,COUNT_MB_C_prev2,COUNT_MB_D_prev2,COUNT_POS_C_prev2,COUNT_POS_D_prev2,custinit_CR_amt_prev2,custinit_DR_amt_prev2,custinit_CR_cnt_prev2,custinit_DR_cnt_prev2,ATM_amt_prev2,ATM_CW_Amt_prev2,ATM_CW_Cnt_prev2,BRN_CW_Amt_prev2,BRN_CW_Cnt_prev2,BRN_CASH_Dep_Amt_prev2,BRN_CASH_Dep_Cnt_prev2,CNR_prev2,BAL_prev2,EOP_prev2,CR_AMB_Prev2,C_prev3,D_prev3,ATM_C_prev3,ATM_D_prev3,BRANCH_C_prev3,BRANCH_D_prev3,IB_C_prev3,IB_D_prev3,MB_C_prev3,MB_D_prev3,POS_C_prev3,POS_D_prev3,count_C_prev3,count_D_prev3,COUNT_ATM_C_prev3,COUNT_ATM_D_prev3,COUNT_BRANCH_C_prev3,COUNT_BRANCH_D_prev3,COUNT_IB_C_prev3,COUNT_IB_D_prev3,COUNT_MB_C_prev3,COUNT_MB_D_prev3,COUNT_POS_C_prev3,COUNT_POS_D_prev3,custinit_CR_amt_prev3,custinit_DR_amt_prev3,custinit_CR_cnt_prev3,custinit_DR_cnt_prev3,ATM_amt_prev3,ATM_CW_Amt_prev3,ATM_CW_Cnt_prev3,BRN_CW_Amt_prev3,BRN_CW_Cnt_prev3,BRN_CASH_Dep_Amt_prev3,BRN_CASH_Dep_Cnt_prev3,CNR_prev3,BAL_prev3,EOP_prev3,CR_AMB_Prev3,C_prev4,D_prev4,ATM_C_prev4,ATM_D_prev4,BRANCH_C_prev4,BRANCH_D_prev4,IB_C_prev4,IB_D_prev4,MB_C_prev4,MB_D_prev4,POS_C_prev4,POS_D_prev4,count_C_prev4,count_D_prev4,COUNT_ATM_C_prev4,COUNT_ATM_D_prev4,COUNT_BRANCH_C_prev4,COUNT_BRANCH_D_prev4,COUNT_IB_C_prev4,COUNT_IB_D_prev4,COUNT_MB_C_prev4,COUNT_MB_D_prev4,COUNT_POS_C_prev4,COUNT_POS_D_prev4,custinit_CR_amt_prev4,custinit_DR_amt_prev4,custinit_CR_cnt_prev4,custinit_DR_cnt_prev4,ATM_amt_prev4,ATM_CW_Amt_prev4,ATM_CW_Cnt_prev4,BRN_CW_Amt_prev4,BRN_CW_Cnt_prev4,BRN_CASH_Dep_Amt_prev4,BRN_CASH_Dep_Cnt_prev4,CNR_prev4,BAL_prev4,EOP_prev4,CR_AMB_Prev4,C_prev5,D_prev5,ATM_C_prev5,ATM_D_prev5,BRANCH_C_prev5,BRANCH_D_prev5,IB_C_prev5,IB_D_prev5,MB_C_prev5,MB_D_prev5,POS_C_prev5,POS_D_prev5,count_C_prev5,count_D_prev5,COUNT_ATM_C_prev5,COUNT_ATM_D_prev5,COUNT_BRANCH_C_prev5,COUNT_BRANCH_D_prev5,COUNT_IB_C_prev5,COUNT_IB_D_prev5,COUNT_MB_C_prev5,COUNT_MB_D_prev5,COUNT_POS_C_prev5,COUNT_POS_D_prev5,custinit_CR_amt_prev5,custinit_DR_amt_prev5,custinit_CR_cnt_prev5,custinit_DR_cnt_prev5,ATM_amt_prev5,ATM_CW_Amt_prev5,ATM_CW_Cnt_prev5,BRN_CW_Amt_prev5,BRN_CW_Cnt_prev5,BRN_CASH_Dep_Amt_prev5,BRN_CASH_Dep_Cnt_prev5,CNR_prev5,BAL_prev5,EOP_prev5,CR_AMB_Prev5,C_prev6,D_prev6,ATM_C_prev6,ATM_D_prev6,BRANCH_C_prev6,BRANCH_D_prev6,IB_C_prev6,IB_D_prev6,MB_C_prev6,MB_D_prev6,POS_C_prev6,POS_D_prev6,count_C_prev6,count_D_prev6,COUNT_ATM_C_prev6,COUNT_ATM_D_prev6,COUNT_BRANCH_C_prev6,COUNT_BRANCH_D_prev6,COUNT_IB_C_prev6,COUNT_IB_D_prev6,COUNT_MB_C_prev6,COUNT_MB_D_prev6,COUNT_POS_C_prev6,COUNT_POS_D_prev6,custinit_CR_amt_prev6,custinit_DR_amt_prev6,custinit_CR_cnt_prev6,custinit_DR_cnt_prev6,ATM_amt_prev6,ATM_CW_Amt_prev6,ATM_CW_Cnt_prev6,BRN_CW_Amt_prev6,BRN_CW_Cnt_prev6,BRN_CASH_Dep_Amt_prev6,BRN_CASH_Dep_Cnt_prev6,CNR_prev6,BAL_prev6,EOP_prev6,CR_AMB_Prev6,FRX_PrevQ1,EFT_SELF_TRANSFER_PrevQ1,Billpay_Active_PrevQ1,Billpay_Reg_ason_Prev1,FD_AMOUNT_BOOK_PrevQ1,FD_AMOUNT_BOOK_PrevQ2,NO_OF_FD_BOOK_PrevQ1,NO_OF_FD_BOOK_PrevQ2,NO_OF_RD_BOOK_PrevQ1,NO_OF_RD_BOOK_PrevQ2,RD_AMOUNT_BOOK_Pr

In [3]:
# Properties of attributes
#print (traindf.city.value_counts(normalize=True))
#print ('Missing % is',(traindf.city.isnull().sum()/len(traindf)))

In [5]:
# Only Y columns
traindf['EMAIL_UNSUBSCRIBE'] = np.where(traindf['EMAIL_UNSUBSCRIBE']=='Y', 'Y', 'N')
traindf['AGRI_TAG_LIVE'] = np.where(traindf['AGRI_TAG_LIVE']=='Y', 1, 0)
traindf['AL_CNC_TAG_LIVE'] = np.where(traindf['AL_CNC_TAG_LIVE']=='Y', 1, 0)
traindf['AL_TAG_LIVE'] = np.where(traindf['AL_TAG_LIVE']=='Y', 1, 0)
traindf['BL_TAG_LIVE'] = np.where(traindf['BL_TAG_LIVE']=='Y', 1, 0)
traindf['CC_TAG_LIVE'] = np.where(traindf['CC_TAG_LIVE']=='Y', 1, 0)
traindf['CE_TAG_LIVE'] = np.where(traindf['CE_TAG_LIVE']=='Y', 1, 0)
traindf['CV_TAG_LIVE'] = np.where(traindf['CV_TAG_LIVE']=='Y', 1, 0)
traindf['DEMAT_TAG_LIVE'] = np.where(traindf['DEMAT_TAG_LIVE']=='Y', 1, 0)
traindf['EDU_TAG_LIVE'] = np.where(traindf['EDU_TAG_LIVE']=='Y', 1, 0)
traindf['GL_TAG_LIVE'] = np.where(traindf['GL_TAG_LIVE']=='Y', 1, 0)
traindf['HL_TAG_LIVE'] = np.where(traindf['HL_TAG_LIVE']=='Y', 1, 0)
traindf['SEC_ACC_TAG_LIVE'] = np.where(traindf['SEC_ACC_TAG_LIVE']=='Y', 1, 0)
traindf['INS_TAG_LIVE'] = np.where(traindf['INS_TAG_LIVE']=='Y', 1, 0)
traindf['LAS_TAG_LIVE'] = np.where(traindf['LAS_TAG_LIVE']=='Y', 1, 0)
traindf['MF_TAG_LIVE'] = np.where(traindf['MF_TAG_LIVE']=='Y', 1, 0)
traindf['OTHER_LOANS_TAG_LIVE'] = np.where(traindf['OTHER_LOANS_TAG_LIVE']=='Y', 1, 0)
traindf['PL_TAG_LIVE'] = np.where(traindf['PL_TAG_LIVE']=='Y', 1, 0)
traindf['RD_TAG_LIVE'] = np.where(traindf['RD_TAG_LIVE']=='Y', 1, 0)
traindf['FD_TAG_LIVE'] = np.where(traindf['FD_TAG_LIVE']=='Y', 1, 0)
traindf['TL_TAG_LIVE'] = np.where(traindf['TL_TAG_LIVE']=='Y', 1, 0)
traindf['TWL_TAG_LIVE'] = np.where(traindf['TWL_TAG_LIVE']=='Y', 1, 0)
traindf['lap_tag_live'] = np.where(traindf['lap_tag_live']=='Y', 1, 0)
traindf['EFT_SELF_TRANSFER_PrevQ1'] = np.where(traindf['EFT_SELF_TRANSFER_PrevQ1']=='Y','Y','N')

In [6]:
#replace missing values
traindf['OCCUP_ALL_NEW'] = traindf['OCCUP_ALL_NEW'].fillna(pd.Series(np.random.choice(['SELF_EMPLOYED','SALARIED','HOUSEWIFE'], p=[0.50,0.30,0.20], size=len(traindf))))
traindf['dependents'] = traindf['dependents'].fillna(pd.Series(np.random.choice([0,2,1], p=[0.85,0.10,0.05], size=len(traindf))))
traindf['FINAL_WORTH_prev1'] = traindf['FINAL_WORTH_prev1'].fillna(pd.Series(np.random.choice(['HIGH','MEDIUM'], p=[0.75,0.25], size=len(traindf))))
traindf['ENGAGEMENT_TAG_prev1'] = traindf['ENGAGEMENT_TAG_prev1'].fillna(pd.Series(np.random.choice(['LOW','HIGH','MEDIUM'], p=[0.50,0.30,0.20], size=len(traindf))))

In [6]:
#traindf['dependents'] = KNN(k=2,verbose=True).complete(traindf['dependents'].values)

In [7]:
del traindf['city'], traindf['zip'], traindf['brn_code']

In [8]:
# New variables
#Diff1
traindf['TAvgCRDiff1'] = ((traindf.fillna(0)['C_prev1']/traindf.fillna(0)['count_C_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['C_prev2']/traindf.fillna(0)['count_C_prev2']).replace(np.inf, 0))
traindf['TAvgDRDiff1'] = ((traindf.fillna(0)['D_prev1']/traindf.fillna(0)['count_D_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['D_prev2']/traindf.fillna(0)['count_C_prev2']).replace(np.inf, 0))
traindf['ATMAvgCRDiff1'] = ((traindf.fillna(0)['ATM_C_prev1']/traindf.fillna(0)['COUNT_ATM_C_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['ATM_C_prev2']/traindf.fillna(0)['COUNT_ATM_C_prev2']).replace(np.inf, 0))
traindf['ATMAvgDRDiff1'] = ((traindf.fillna(0)['ATM_D_prev1']/traindf.fillna(0)['COUNT_ATM_D_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['ATM_D_prev2']/traindf.fillna(0)['COUNT_ATM_D_prev2']).replace(np.inf, 0))
traindf['BRAvgCRDiff1'] = ((traindf.fillna(0)['BRANCH_C_prev1']/traindf.fillna(0)['COUNT_BRANCH_C_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['BRANCH_C_prev2']/traindf.fillna(0)['COUNT_BRANCH_C_prev2']).replace(np.inf, 0))
traindf['BRAvgDRDiff1'] = ((traindf.fillna(0)['BRANCH_D_prev1']/traindf.fillna(0)['COUNT_BRANCH_D_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['BRANCH_D_prev2']/traindf.fillna(0)['COUNT_BRANCH_D_prev2']).replace(np.inf, 0))
traindf['IBAvgCRDiff1'] = ((traindf.fillna(0)['IB_C_prev1']/traindf.fillna(0)['COUNT_IB_C_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['IB_C_prev2']/traindf.fillna(0)['COUNT_IB_C_prev2']).replace(np.inf, 0))
traindf['IBAvgDRDiff1'] = ((traindf.fillna(0)['IB_D_prev1']/traindf.fillna(0)['COUNT_IB_D_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['IB_D_prev2']/traindf.fillna(0)['COUNT_IB_D_prev2']).replace(np.inf, 0))
traindf['MBAvgCRDiff1'] = ((traindf.fillna(0)['MB_C_prev1']/traindf.fillna(0)['COUNT_MB_C_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['MB_C_prev2']/traindf.fillna(0)['COUNT_MB_C_prev2']).replace(np.inf, 0))
traindf['MBAvgDRDiff1'] = ((traindf.fillna(0)['MB_D_prev1']/traindf.fillna(0)['COUNT_MB_D_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['MB_D_prev2']/traindf.fillna(0)['COUNT_MB_D_prev2']).replace(np.inf, 0))
traindf['POSAvgCRDiff1'] = ((traindf.fillna(0)['POS_C_prev1']/traindf.fillna(0)['COUNT_POS_C_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['POS_C_prev2']/traindf.fillna(0)['COUNT_POS_C_prev2']).replace(np.inf, 0))
traindf['POSAvgDRDiff1'] = ((traindf.fillna(0)['POS_D_prev1']/traindf.fillna(0)['COUNT_POS_D_prev1']).replace(np.inf, 0)) - ((traindf.fillna(0)['POS_D_prev2']/traindf.fillna(0)['COUNT_POS_D_prev2']).replace(np.inf, 0))

In [9]:
#Diff2
traindf['TAvgCRDiff2'] = ((traindf.fillna(0)['C_prev2']/traindf.fillna(0)['count_C_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['C_prev3']/traindf.fillna(0)['count_C_prev3']).replace(np.inf, 0))
traindf['TAvgDRDiff2'] = ((traindf.fillna(0)['D_prev2']/traindf.fillna(0)['count_D_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['D_prev3']/traindf.fillna(0)['count_C_prev3']).replace(np.inf, 0))
traindf['ATMAvgCRDiff2'] = ((traindf.fillna(0)['ATM_C_prev2']/traindf.fillna(0)['COUNT_ATM_C_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['ATM_C_prev3']/traindf.fillna(0)['COUNT_ATM_C_prev3']).replace(np.inf, 0))
traindf['ATMAvgDRDiff2'] = ((traindf.fillna(0)['ATM_D_prev2']/traindf.fillna(0)['COUNT_ATM_D_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['ATM_D_prev3']/traindf.fillna(0)['COUNT_ATM_D_prev3']).replace(np.inf, 0))
traindf['BRAvgCRDiff2'] = ((traindf.fillna(0)['BRANCH_C_prev2']/traindf.fillna(0)['COUNT_BRANCH_C_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['BRANCH_C_prev3']/traindf.fillna(0)['COUNT_BRANCH_C_prev3']).replace(np.inf, 0))
traindf['BRAvgDRDiff2'] = ((traindf.fillna(0)['BRANCH_D_prev2']/traindf.fillna(0)['COUNT_BRANCH_D_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['BRANCH_D_prev3']/traindf.fillna(0)['COUNT_BRANCH_D_prev3']).replace(np.inf, 0))
traindf['IBAvgCRDiff2'] = ((traindf.fillna(0)['IB_C_prev2']/traindf.fillna(0)['COUNT_IB_C_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['IB_C_prev3']/traindf.fillna(0)['COUNT_IB_C_prev3']).replace(np.inf, 0))
traindf['IBAvgDRDiff2'] = ((traindf.fillna(0)['IB_D_prev2']/traindf.fillna(0)['COUNT_IB_D_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['IB_D_prev3']/traindf.fillna(0)['COUNT_IB_D_prev3']).replace(np.inf, 0))
traindf['MBAvgCRDiff2'] = ((traindf.fillna(0)['MB_C_prev2']/traindf.fillna(0)['COUNT_MB_C_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['MB_C_prev3']/traindf.fillna(0)['COUNT_MB_C_prev3']).replace(np.inf, 0))
traindf['MBAvgDRDiff2'] = ((traindf.fillna(0)['MB_D_prev2']/traindf.fillna(0)['COUNT_MB_D_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['MB_D_prev3']/traindf.fillna(0)['COUNT_MB_D_prev3']).replace(np.inf, 0))
traindf['POSAvgCRDiff2'] = ((traindf.fillna(0)['POS_C_prev2']/traindf.fillna(0)['COUNT_POS_C_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['POS_C_prev3']/traindf.fillna(0)['COUNT_POS_C_prev3']).replace(np.inf, 0))
traindf['POSAvgDRDiff2'] = ((traindf.fillna(0)['POS_D_prev2']/traindf.fillna(0)['COUNT_POS_D_prev2']).replace(np.inf, 0)) - ((traindf.fillna(0)['POS_D_prev3']/traindf.fillna(0)['COUNT_POS_D_prev3']).replace(np.inf, 0))

In [10]:
#Diff3
traindf['TAvgCRDiff3'] = ((traindf.fillna(0)['C_prev3']/traindf.fillna(0)['count_C_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['C_prev4']/traindf.fillna(0)['count_C_prev4']).replace(np.inf, 0))
traindf['TAvgDRDiff3'] = ((traindf.fillna(0)['D_prev3']/traindf.fillna(0)['count_D_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['D_prev4']/traindf.fillna(0)['count_C_prev4']).replace(np.inf, 0))
traindf['ATMAvgCRDiff3'] = ((traindf.fillna(0)['ATM_C_prev3']/traindf.fillna(0)['COUNT_ATM_C_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['ATM_C_prev4']/traindf.fillna(0)['COUNT_ATM_C_prev4']).replace(np.inf, 0))
traindf['ATMAvgDRDiff3'] = ((traindf.fillna(0)['ATM_D_prev3']/traindf.fillna(0)['COUNT_ATM_D_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['ATM_D_prev4']/traindf.fillna(0)['COUNT_ATM_D_prev4']).replace(np.inf, 0))
traindf['BRAvgCRDiff3'] = ((traindf.fillna(0)['BRANCH_C_prev3']/traindf.fillna(0)['COUNT_BRANCH_C_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['BRANCH_C_prev4']/traindf.fillna(0)['COUNT_BRANCH_C_prev4']).replace(np.inf, 0))
traindf['BRAvgDRDiff3'] = ((traindf.fillna(0)['BRANCH_D_prev3']/traindf.fillna(0)['COUNT_BRANCH_D_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['BRANCH_D_prev4']/traindf.fillna(0)['COUNT_BRANCH_D_prev4']).replace(np.inf, 0))
traindf['IBAvgCRDiff3'] = ((traindf.fillna(0)['IB_C_prev3']/traindf.fillna(0)['COUNT_IB_C_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['IB_C_prev4']/traindf.fillna(0)['COUNT_IB_C_prev4']).replace(np.inf, 0))
traindf['IBAvgDRDiff3'] = ((traindf.fillna(0)['IB_D_prev3']/traindf.fillna(0)['COUNT_IB_D_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['IB_D_prev4']/traindf.fillna(0)['COUNT_IB_D_prev4']).replace(np.inf, 0))
traindf['MBAvgCRDiff3'] = ((traindf.fillna(0)['MB_C_prev3']/traindf.fillna(0)['COUNT_MB_C_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['MB_C_prev4']/traindf.fillna(0)['COUNT_MB_C_prev4']).replace(np.inf, 0))
traindf['MBAvgDRDiff3'] = ((traindf.fillna(0)['MB_D_prev3']/traindf.fillna(0)['COUNT_MB_D_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['MB_D_prev4']/traindf.fillna(0)['COUNT_MB_D_prev4']).replace(np.inf, 0))
traindf['POSAvgCRDiff3'] = ((traindf.fillna(0)['POS_C_prev3']/traindf.fillna(0)['COUNT_POS_C_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['POS_C_prev4']/traindf.fillna(0)['COUNT_POS_C_prev4']).replace(np.inf, 0))
traindf['POSAvgDRDiff3'] = ((traindf.fillna(0)['POS_D_prev3']/traindf.fillna(0)['COUNT_POS_D_prev3']).replace(np.inf, 0)) - ((traindf.fillna(0)['POS_D_prev4']/traindf.fillna(0)['COUNT_POS_D_prev4']).replace(np.inf, 0))

In [11]:
#Diff4
traindf['TAvgCRDiff4'] = ((traindf.fillna(0)['C_prev4']/traindf.fillna(0)['count_C_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['C_prev5']/traindf.fillna(0)['count_C_prev5']).replace(np.inf, 0))
traindf['TAvgDRDiff4'] = ((traindf.fillna(0)['D_prev4']/traindf.fillna(0)['count_D_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['D_prev5']/traindf.fillna(0)['count_C_prev5']).replace(np.inf, 0))
traindf['ATMAvgCRDiff4'] = ((traindf.fillna(0)['ATM_C_prev4']/traindf.fillna(0)['COUNT_ATM_C_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['ATM_C_prev5']/traindf.fillna(0)['COUNT_ATM_C_prev5']).replace(np.inf, 0))
traindf['ATMAvgDRDiff4'] = ((traindf.fillna(0)['ATM_D_prev4']/traindf.fillna(0)['COUNT_ATM_D_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['ATM_D_prev5']/traindf.fillna(0)['COUNT_ATM_D_prev5']).replace(np.inf, 0))
traindf['BRAvgCRDiff4'] = ((traindf.fillna(0)['BRANCH_C_prev4']/traindf.fillna(0)['COUNT_BRANCH_C_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['BRANCH_C_prev5']/traindf.fillna(0)['COUNT_BRANCH_C_prev5']).replace(np.inf, 0))
traindf['BRAvgDRDiff4'] = ((traindf.fillna(0)['BRANCH_D_prev4']/traindf.fillna(0)['COUNT_BRANCH_D_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['BRANCH_D_prev5']/traindf.fillna(0)['COUNT_BRANCH_D_prev5']).replace(np.inf, 0))
traindf['IBAvgCRDiff4'] = ((traindf.fillna(0)['IB_C_prev4']/traindf.fillna(0)['COUNT_IB_C_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['IB_C_prev5']/traindf.fillna(0)['COUNT_IB_C_prev5']).replace(np.inf, 0))
traindf['IBAvgDRDiff4'] = ((traindf.fillna(0)['IB_D_prev4']/traindf.fillna(0)['COUNT_IB_D_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['IB_D_prev5']/traindf.fillna(0)['COUNT_IB_D_prev5']).replace(np.inf, 0))
traindf['MBAvgCRDiff4'] = ((traindf.fillna(0)['MB_C_prev4']/traindf.fillna(0)['COUNT_MB_C_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['MB_C_prev5']/traindf.fillna(0)['COUNT_MB_C_prev5']).replace(np.inf, 0))
traindf['MBAvgDRDiff4'] = ((traindf.fillna(0)['MB_D_prev4']/traindf.fillna(0)['COUNT_MB_D_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['MB_D_prev5']/traindf.fillna(0)['COUNT_MB_D_prev5']).replace(np.inf, 0))
traindf['POSAvgCRDiff4'] = ((traindf.fillna(0)['POS_C_prev4']/traindf.fillna(0)['COUNT_POS_C_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['POS_C_prev5']/traindf.fillna(0)['COUNT_POS_C_prev5']).replace(np.inf, 0))
traindf['POSAvgDRDiff4'] = ((traindf.fillna(0)['POS_D_prev4']/traindf.fillna(0)['COUNT_POS_D_prev4']).replace(np.inf, 0)) - ((traindf.fillna(0)['POS_D_prev5']/traindf.fillna(0)['COUNT_POS_D_prev5']).replace(np.inf, 0))

In [12]:
#Diff5
traindf['TAvgCRDiff5'] = ((traindf.fillna(0)['C_prev5']/traindf.fillna(0)['count_C_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['C_prev6']/traindf.fillna(0)['count_C_prev6']).replace(np.inf, 0))
traindf['TAvgDRDiff5'] = ((traindf.fillna(0)['D_prev5']/traindf.fillna(0)['count_D_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['D_prev6']/traindf.fillna(0)['count_C_prev6']).replace(np.inf, 0))
traindf['ATMAvgCRDiff5'] = ((traindf.fillna(0)['ATM_C_prev5']/traindf.fillna(0)['COUNT_ATM_C_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['ATM_C_prev6']/traindf.fillna(0)['COUNT_ATM_C_prev6']).replace(np.inf, 0))
traindf['ATMAvgDRDiff5'] = ((traindf.fillna(0)['ATM_D_prev5']/traindf.fillna(0)['COUNT_ATM_D_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['ATM_D_prev6']/traindf.fillna(0)['COUNT_ATM_D_prev6']).replace(np.inf, 0))
traindf['BRAvgCRDiff5'] = ((traindf.fillna(0)['BRANCH_C_prev5']/traindf.fillna(0)['COUNT_BRANCH_C_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['BRANCH_C_prev6']/traindf.fillna(0)['COUNT_BRANCH_C_prev6']).replace(np.inf, 0))
traindf['BRAvgDRDiff5'] = ((traindf.fillna(0)['BRANCH_D_prev5']/traindf.fillna(0)['COUNT_BRANCH_D_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['BRANCH_D_prev6']/traindf.fillna(0)['COUNT_BRANCH_D_prev6']).replace(np.inf, 0))
traindf['IBAvgCRDiff5'] = ((traindf.fillna(0)['IB_C_prev5']/traindf.fillna(0)['COUNT_IB_C_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['IB_C_prev6']/traindf.fillna(0)['COUNT_IB_C_prev6']).replace(np.inf, 0))
traindf['IBAvgDRDiff5'] = ((traindf.fillna(0)['IB_D_prev5']/traindf.fillna(0)['COUNT_IB_D_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['IB_D_prev6']/traindf.fillna(0)['COUNT_IB_D_prev6']).replace(np.inf, 0))
traindf['MBAvgCRDiff5'] = ((traindf.fillna(0)['MB_C_prev5']/traindf.fillna(0)['COUNT_MB_C_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['MB_C_prev6']/traindf.fillna(0)['COUNT_MB_C_prev6']).replace(np.inf, 0))
traindf['MBAvgDRDiff5'] = ((traindf.fillna(0)['MB_D_prev5']/traindf.fillna(0)['COUNT_MB_D_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['MB_D_prev6']/traindf.fillna(0)['COUNT_MB_D_prev6']).replace(np.inf, 0))
traindf['POSAvgCRDiff5'] = ((traindf.fillna(0)['POS_C_prev5']/traindf.fillna(0)['COUNT_POS_C_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['POS_C_prev6']/traindf.fillna(0)['COUNT_POS_C_prev6']).replace(np.inf, 0))
traindf['POSAvgDRDiff5'] = ((traindf.fillna(0)['POS_D_prev5']/traindf.fillna(0)['COUNT_POS_D_prev5']).replace(np.inf, 0)) - ((traindf.fillna(0)['POS_D_prev6']/traindf.fillna(0)['COUNT_POS_D_prev6']).replace(np.inf, 0))

In [13]:
# Delete 1
del traindf['C_prev1'],traindf['D_prev1'],traindf['ATM_C_prev1'],traindf['ATM_D_prev1'],traindf['BRANCH_C_prev1']
del traindf['BRANCH_D_prev1'],traindf['IB_C_prev1'],traindf['IB_D_prev1'],traindf['MB_C_prev1'],traindf['MB_D_prev1']
del traindf['POS_C_prev1'],traindf['POS_D_prev1'],traindf['count_C_prev1'],traindf['count_D_prev1'],traindf['COUNT_ATM_C_prev1']
del traindf['COUNT_ATM_D_prev1'],traindf['COUNT_BRANCH_C_prev1'],traindf['COUNT_BRANCH_D_prev1'],traindf['COUNT_IB_C_prev1']
del traindf['COUNT_IB_D_prev1'],traindf['COUNT_MB_C_prev1'],traindf['COUNT_MB_D_prev1'],traindf['COUNT_POS_C_prev1'],traindf['COUNT_POS_D_prev1']
# Delete 2
del traindf['C_prev2'],traindf['D_prev2'],traindf['ATM_C_prev2'],traindf['ATM_D_prev2'],traindf['BRANCH_C_prev2']
del traindf['BRANCH_D_prev2'],traindf['IB_C_prev2'],traindf['IB_D_prev2'],traindf['MB_C_prev2'],traindf['MB_D_prev2']
del traindf['POS_C_prev2'],traindf['POS_D_prev2'],traindf['count_C_prev2'],traindf['count_D_prev2'],traindf['COUNT_ATM_C_prev2']
del traindf['COUNT_ATM_D_prev2'],traindf['COUNT_BRANCH_C_prev2'],traindf['COUNT_BRANCH_D_prev2'],traindf['COUNT_IB_C_prev2']
del traindf['COUNT_IB_D_prev2'],traindf['COUNT_MB_C_prev2'],traindf['COUNT_MB_D_prev2'],traindf['COUNT_POS_C_prev2'],traindf['COUNT_POS_D_prev2']
# Delete 3
del traindf['C_prev3'],traindf['D_prev3'],traindf['ATM_C_prev3'],traindf['ATM_D_prev3'],traindf['BRANCH_C_prev3']
del traindf['BRANCH_D_prev3'],traindf['IB_C_prev3'],traindf['IB_D_prev3'],traindf['MB_C_prev3'],traindf['MB_D_prev3']
del traindf['POS_C_prev3'],traindf['POS_D_prev3'],traindf['count_C_prev3'],traindf['count_D_prev3'],traindf['COUNT_ATM_C_prev3']
del traindf['COUNT_ATM_D_prev3'],traindf['COUNT_BRANCH_C_prev3'],traindf['COUNT_BRANCH_D_prev3'],traindf['COUNT_IB_C_prev3']
del traindf['COUNT_IB_D_prev3'],traindf['COUNT_MB_C_prev3'],traindf['COUNT_MB_D_prev3'],traindf['COUNT_POS_C_prev3'],traindf['COUNT_POS_D_prev3']
# Delete 4
del traindf['C_prev4'],traindf['D_prev4'],traindf['ATM_C_prev4'],traindf['ATM_D_prev4'],traindf['BRANCH_C_prev4']
del traindf['BRANCH_D_prev4'],traindf['IB_C_prev4'],traindf['IB_D_prev4'],traindf['MB_C_prev4'],traindf['MB_D_prev4']
del traindf['POS_C_prev4'],traindf['POS_D_prev4'],traindf['count_C_prev4'],traindf['count_D_prev4'],traindf['COUNT_ATM_C_prev4']
del traindf['COUNT_ATM_D_prev4'],traindf['COUNT_BRANCH_C_prev4'],traindf['COUNT_BRANCH_D_prev4'],traindf['COUNT_IB_C_prev4']
del traindf['COUNT_IB_D_prev4'],traindf['COUNT_MB_C_prev4'],traindf['COUNT_MB_D_prev4'],traindf['COUNT_POS_C_prev4'],traindf['COUNT_POS_D_prev4']
# Delete 5
del traindf['C_prev5'],traindf['D_prev5'],traindf['ATM_C_prev5'],traindf['ATM_D_prev5'],traindf['BRANCH_C_prev5']
del traindf['BRANCH_D_prev5'],traindf['IB_C_prev5'],traindf['IB_D_prev5'],traindf['MB_C_prev5'],traindf['MB_D_prev5']
del traindf['POS_C_prev5'],traindf['POS_D_prev5'],traindf['count_C_prev5'],traindf['count_D_prev5'],traindf['COUNT_ATM_C_prev5']
del traindf['COUNT_ATM_D_prev5'],traindf['COUNT_BRANCH_C_prev5'],traindf['COUNT_BRANCH_D_prev5'],traindf['COUNT_IB_C_prev5']
del traindf['COUNT_IB_D_prev5'],traindf['COUNT_MB_C_prev5'],traindf['COUNT_MB_D_prev5'],traindf['COUNT_POS_C_prev5'],traindf['COUNT_POS_D_prev5']
# Delete 6
del traindf['C_prev6'],traindf['D_prev6'],traindf['ATM_C_prev6'],traindf['ATM_D_prev6'],traindf['BRANCH_C_prev6']
del traindf['BRANCH_D_prev6'],traindf['IB_C_prev6'],traindf['IB_D_prev6'],traindf['MB_C_prev6'],traindf['MB_D_prev6']
del traindf['POS_C_prev6'],traindf['POS_D_prev6'],traindf['count_C_prev6'],traindf['count_D_prev6'],traindf['COUNT_ATM_C_prev6']
del traindf['COUNT_ATM_D_prev6'],traindf['COUNT_BRANCH_C_prev6'],traindf['COUNT_BRANCH_D_prev6'],traindf['COUNT_IB_C_prev6']
del traindf['COUNT_IB_D_prev6'],traindf['COUNT_MB_C_prev6'],traindf['COUNT_MB_D_prev6'],traindf['COUNT_POS_C_prev6'],traindf['COUNT_POS_D_prev6']

In [14]:
traindf.shape

(300000, 290)

In [15]:
# Create New variables
# Set1
traindf['CuIniCRDiff1'] = ((traindf.fillna(0)['custinit_CR_amt_prev1']/traindf.fillna(0)['custinit_CR_cnt_prev1'])-
                           (traindf.fillna(0)['custinit_CR_amt_prev2']/traindf.fillna(0)['custinit_CR_cnt_prev2']))
traindf['CuIniDRDiff1'] = ((traindf.fillna(0)['custinit_DR_amt_prev1']/traindf.fillna(0)['custinit_DR_cnt_prev1'])-
                           (traindf.fillna(0)['custinit_DR_amt_prev2']/traindf.fillna(0)['custinit_DR_cnt_prev2']))
traindf['ATMWithDiff1'] = (traindf.fillna(0)['ATM_amt_prev1'] - traindf.fillna(0)['ATM_amt_prev2'])
traindf['ATMAvDwDiff1'] = (((traindf.fillna(0)['ATM_CW_Amt_prev1']/traindf.fillna(0)['ATM_CW_Cnt_prev1']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['ATM_CW_Amt_prev2']/traindf.fillna(0)['ATM_CW_Cnt_prev2']).replace(np.inf, 0)))
traindf['BCWAvDwDiff1'] = (((traindf.fillna(0)['BRN_CW_Amt_prev1']/traindf.fillna(0)['BRN_CW_Cnt_prev1']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['BRN_CW_Amt_prev2']/traindf.fillna(0)['BRN_CW_Cnt_prev2']).replace(np.inf, 0)))
traindf['BCDAvDwDiff1'] = (((traindf.fillna(0)['BRN_CASH_Dep_Amt_prev1']/traindf.fillna(0)['BRN_CASH_Dep_Cnt_prev1']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['BRN_CASH_Dep_Amt_prev2']/traindf.fillna(0)['BRN_CASH_Dep_Cnt_prev2']).replace(np.inf, 0)))
traindf['CNTDiff1'] = (traindf.fillna(0)['CNR_prev1'] - traindf.fillna(0)['CNR_prev2'])
traindf['BALDiff1'] = (traindf.fillna(0)['BAL_prev1'] - traindf.fillna(0)['BAL_prev2'])
traindf['EOPDiff1'] = (traindf.fillna(0)['EOP_prev1'] - traindf.fillna(0)['EOP_prev2'])
traindf['CRAMBDiff1'] = (traindf.fillna(0)['CR_AMB_Prev1'] - traindf.fillna(0)['CR_AMB_Prev2'])

In [16]:
# Set 2
traindf['CuIniCRDiff2'] = ((traindf.fillna(0)['custinit_CR_amt_prev2']/traindf.fillna(0)['custinit_CR_cnt_prev2'])-
                           (traindf.fillna(0)['custinit_CR_amt_prev3']/traindf.fillna(0)['custinit_CR_cnt_prev3']))
traindf['CuIniDRDiff2'] = ((traindf.fillna(0)['custinit_DR_amt_prev2']/traindf.fillna(0)['custinit_DR_cnt_prev2'])-
                           (traindf.fillna(0)['custinit_DR_amt_prev3']/traindf.fillna(0)['custinit_DR_cnt_prev3']))
traindf['ATMWithDiff2'] = (traindf.fillna(0)['ATM_amt_prev2'] - traindf.fillna(0)['ATM_amt_prev3'])
traindf['ATMAvDwDiff2'] = (((traindf.fillna(0)['ATM_CW_Amt_prev2']/traindf.fillna(0)['ATM_CW_Cnt_prev2']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['ATM_CW_Amt_prev3']/traindf.fillna(0)['ATM_CW_Cnt_prev3']).replace(np.inf, 0)))
traindf['BCWAvDwDiff2'] = (((traindf.fillna(0)['BRN_CW_Amt_prev2']/traindf.fillna(0)['BRN_CW_Cnt_prev2']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['BRN_CW_Amt_prev3']/traindf.fillna(0)['BRN_CW_Cnt_prev3']).replace(np.inf, 0)))
traindf['BCDAvDwDiff2'] = (((traindf.fillna(0)['BRN_CASH_Dep_Amt_prev2']/traindf.fillna(0)['BRN_CASH_Dep_Cnt_prev2']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['BRN_CASH_Dep_Amt_prev3']/traindf.fillna(0)['BRN_CASH_Dep_Cnt_prev3']).replace(np.inf, 0)))
traindf['CNTDiff2'] = (traindf.fillna(0)['CNR_prev2'] - traindf.fillna(0)['CNR_prev3'])
traindf['BALDiff2'] = (traindf.fillna(0)['BAL_prev2'] - traindf.fillna(0)['BAL_prev3'])
traindf['EOPDiff2'] = (traindf.fillna(0)['EOP_prev2'] - traindf.fillna(0)['EOP_prev3'])
traindf['CRAMBDiff2'] = (traindf.fillna(0)['CR_AMB_Prev2'] - traindf.fillna(0)['CR_AMB_Prev3'])

In [17]:
# Set 3
traindf['CuIniCRDiff3'] = ((traindf.fillna(0)['custinit_CR_amt_prev3']/traindf.fillna(0)['custinit_CR_cnt_prev3'])-
                           (traindf.fillna(0)['custinit_CR_amt_prev4']/traindf.fillna(0)['custinit_CR_cnt_prev4']))
traindf['CuIniDRDiff3'] = ((traindf.fillna(0)['custinit_DR_amt_prev3']/traindf.fillna(0)['custinit_DR_cnt_prev3'])-
                           (traindf.fillna(0)['custinit_DR_amt_prev4']/traindf.fillna(0)['custinit_DR_cnt_prev4']))
traindf['ATMWithDiff3'] = (traindf.fillna(0)['ATM_amt_prev3'] - traindf.fillna(0)['ATM_amt_prev4'])
traindf['ATMAvDwDiff3'] = (((traindf.fillna(0)['ATM_CW_Amt_prev3']/traindf.fillna(0)['ATM_CW_Cnt_prev3']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['ATM_CW_Amt_prev4']/traindf.fillna(0)['ATM_CW_Cnt_prev4']).replace(np.inf, 0)))
traindf['BCWAvDwDiff3'] = (((traindf.fillna(0)['BRN_CW_Amt_prev3']/traindf.fillna(0)['BRN_CW_Cnt_prev3']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['BRN_CW_Amt_prev4']/traindf.fillna(0)['BRN_CW_Cnt_prev4']).replace(np.inf, 0)))
traindf['BCDAvDwDiff3'] = (((traindf.fillna(0)['BRN_CASH_Dep_Amt_prev3']/traindf.fillna(0)['BRN_CASH_Dep_Cnt_prev3']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['BRN_CASH_Dep_Amt_prev4']/traindf.fillna(0)['BRN_CASH_Dep_Cnt_prev4']).replace(np.inf, 0)))
traindf['CNTDiff3'] = (traindf.fillna(0)['CNR_prev3'] - traindf.fillna(0)['CNR_prev4'])
traindf['BALDiff3'] = (traindf.fillna(0)['BAL_prev3'] - traindf.fillna(0)['BAL_prev4'])
traindf['EOPDiff3'] = (traindf.fillna(0)['EOP_prev3'] - traindf.fillna(0)['EOP_prev4'])
traindf['CRAMBDiff3'] = (traindf.fillna(0)['CR_AMB_Prev3'] - traindf.fillna(0)['CR_AMB_Prev4'])

In [18]:
# Set 4
traindf['CuIniCRDiff4'] = ((traindf.fillna(0)['custinit_CR_amt_prev4']/traindf.fillna(0)['custinit_CR_cnt_prev4'])- 
                           (traindf.fillna(0)['custinit_CR_amt_prev5']/traindf.fillna(0)['custinit_CR_cnt_prev5']))
traindf['CuIniDRDiff4'] = ((traindf.fillna(0)['custinit_DR_amt_prev4']/traindf.fillna(0)['custinit_DR_cnt_prev4'])- 
                           (traindf.fillna(0)['custinit_DR_amt_prev5']/traindf.fillna(0)['custinit_DR_cnt_prev5']))
traindf['ATMWithDiff4'] = (traindf.fillna(0)['ATM_amt_prev4'] - traindf.fillna(0)['ATM_amt_prev5'])
traindf['ATMAvDwDiff4'] = (((traindf.fillna(0)['ATM_CW_Amt_prev4']/traindf.fillna(0)['ATM_CW_Cnt_prev4']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['ATM_CW_Amt_prev5']/traindf.fillna(0)['ATM_CW_Cnt_prev5']).replace(np.inf, 0)))
traindf['BCWAvDwDiff4'] = (((traindf.fillna(0)['BRN_CW_Amt_prev4']/traindf.fillna(0)['BRN_CW_Cnt_prev4']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['BRN_CW_Amt_prev5']/traindf.fillna(0)['BRN_CW_Cnt_prev5']).replace(np.inf, 0)))
traindf['BCDAvDwDiff4'] = (((traindf.fillna(0)['BRN_CASH_Dep_Amt_prev4']/traindf.fillna(0)['BRN_CASH_Dep_Cnt_prev4']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['BRN_CASH_Dep_Amt_prev5']/traindf.fillna(0)['BRN_CASH_Dep_Cnt_prev5']).replace(np.inf, 0)))
traindf['CNTDiff4'] = (traindf.fillna(0)['CNR_prev4'] - traindf.fillna(0)['CNR_prev5'])
traindf['BALDiff4'] = (traindf.fillna(0)['BAL_prev4'] - traindf.fillna(0)['BAL_prev5'])
traindf['EOPDiff4'] = (traindf.fillna(0)['EOP_prev4'] - traindf.fillna(0)['EOP_prev5'])
traindf['CRAMBDiff4'] = (traindf.fillna(0)['CR_AMB_Prev4'] - traindf.fillna(0)['CR_AMB_Prev5'])

In [19]:
# Set 5
traindf['CuIniCRDiff5'] = ((traindf.fillna(0)['custinit_CR_amt_prev5']/traindf.fillna(0)['custinit_CR_cnt_prev5'])-
                           (traindf.fillna(0)['custinit_CR_amt_prev6']/traindf.fillna(0)['custinit_CR_cnt_prev6']))
traindf['CuIniDRDiff5'] = ((traindf.fillna(0)['custinit_DR_amt_prev5']/traindf.fillna(0)['custinit_DR_cnt_prev5'])-
                           (traindf.fillna(0)['custinit_DR_amt_prev6']/traindf.fillna(0)['custinit_DR_cnt_prev6']))
traindf['ATMWithDiff5'] = (traindf.fillna(0)['ATM_amt_prev5'] - traindf.fillna(0)['ATM_amt_prev6'])
traindf['ATMAvDwDiff5'] = (((traindf.fillna(0)['ATM_CW_Amt_prev5']/traindf.fillna(0)['ATM_CW_Cnt_prev5']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['ATM_CW_Amt_prev6']/traindf.fillna(0)['ATM_CW_Cnt_prev6']).replace(np.inf, 0)))
traindf['BCWAvDwDiff5'] = (((traindf.fillna(0)['BRN_CW_Amt_prev5']/traindf.fillna(0)['BRN_CW_Cnt_prev5']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['BRN_CW_Amt_prev6']/traindf.fillna(0)['BRN_CW_Cnt_prev6']).replace(np.inf, 0)))
traindf['BCDAvDwDiff5'] = (((traindf.fillna(0)['BRN_CASH_Dep_Amt_prev5']/traindf.fillna(0)['BRN_CASH_Dep_Cnt_prev5']).replace(np.inf, 0))-
                           ((traindf.fillna(0)['BRN_CASH_Dep_Amt_prev6']/traindf.fillna(0)['BRN_CASH_Dep_Cnt_prev6']).replace(np.inf, 0)))
traindf['CNTDiff5'] = (traindf.fillna(0)['CNR_prev5'] - traindf.fillna(0)['CNR_prev6'])
traindf['BALDiff5'] = (traindf.fillna(0)['BAL_prev5'] - traindf.fillna(0)['BAL_prev6'])
traindf['EOPDiff5'] = (traindf.fillna(0)['EOP_prev5'] - traindf.fillna(0)['EOP_prev6'])
traindf['CRAMBDiff5'] = (traindf.fillna(0)['CR_AMB_Prev5'] - traindf.fillna(0)['CR_AMB_Prev6'])

In [20]:
# Delete the old columns
# Delete 1
del traindf['custinit_CR_amt_prev1'],traindf['custinit_DR_amt_prev1'],traindf['custinit_CR_cnt_prev1'],traindf['custinit_DR_cnt_prev1']
del traindf['ATM_amt_prev1'],traindf['ATM_CW_Amt_prev1'],traindf['ATM_CW_Cnt_prev1'],traindf['BRN_CW_Amt_prev1']
del traindf['BRN_CW_Cnt_prev1'],traindf['BRN_CASH_Dep_Amt_prev1'],traindf['BRN_CASH_Dep_Cnt_prev1'],traindf['CNR_prev1']
del traindf['BAL_prev1'],traindf['EOP_prev1'],traindf['CR_AMB_Prev1']
# Delete 2
del traindf['custinit_CR_amt_prev2'],traindf['custinit_DR_amt_prev2'],traindf['custinit_CR_cnt_prev2'],traindf['custinit_DR_cnt_prev2']
del traindf['ATM_amt_prev2'],traindf['ATM_CW_Amt_prev2'],traindf['ATM_CW_Cnt_prev2'],traindf['BRN_CW_Amt_prev2']
del traindf['BRN_CW_Cnt_prev2'],traindf['BRN_CASH_Dep_Amt_prev2'],traindf['BRN_CASH_Dep_Cnt_prev2'],traindf['CNR_prev2']
del traindf['BAL_prev2'],traindf['EOP_prev2'],traindf['CR_AMB_Prev2']
# Delete 3
del traindf['custinit_CR_amt_prev3'],traindf['custinit_DR_amt_prev3'],traindf['custinit_CR_cnt_prev3'],traindf['custinit_DR_cnt_prev3']
del traindf['ATM_amt_prev3'],traindf['ATM_CW_Amt_prev3'],traindf['ATM_CW_Cnt_prev3'],traindf['BRN_CW_Amt_prev3']
del traindf['BRN_CW_Cnt_prev3'],traindf['BRN_CASH_Dep_Amt_prev3'],traindf['BRN_CASH_Dep_Cnt_prev3'],traindf['CNR_prev3']
del traindf['BAL_prev3'],traindf['EOP_prev3'],traindf['CR_AMB_Prev3']
# Delete 4
del traindf['custinit_CR_amt_prev4'],traindf['custinit_DR_amt_prev4'],traindf['custinit_CR_cnt_prev4'],traindf['custinit_DR_cnt_prev4']
del traindf['ATM_amt_prev4'],traindf['ATM_CW_Amt_prev4'],traindf['ATM_CW_Cnt_prev4'],traindf['BRN_CW_Amt_prev4']
del traindf['BRN_CW_Cnt_prev4'],traindf['BRN_CASH_Dep_Amt_prev4'],traindf['BRN_CASH_Dep_Cnt_prev4'],traindf['CNR_prev4']
del traindf['BAL_prev4'],traindf['EOP_prev4'],traindf['CR_AMB_Prev4']
# Delete 5
del traindf['custinit_CR_amt_prev5'],traindf['custinit_DR_amt_prev5'],traindf['custinit_CR_cnt_prev5'],traindf['custinit_DR_cnt_prev5']
del traindf['ATM_amt_prev5'],traindf['ATM_CW_Amt_prev5'],traindf['ATM_CW_Cnt_prev5'],traindf['BRN_CW_Amt_prev5']
del traindf['BRN_CW_Cnt_prev5'],traindf['BRN_CASH_Dep_Amt_prev5'],traindf['BRN_CASH_Dep_Cnt_prev5'],traindf['CNR_prev5']
del traindf['BAL_prev5'],traindf['EOP_prev5'],traindf['CR_AMB_Prev5']
# Delete 6
del traindf['custinit_CR_amt_prev6'],traindf['custinit_DR_amt_prev6'],traindf['custinit_CR_cnt_prev6'],traindf['custinit_DR_cnt_prev6']
del traindf['ATM_amt_prev6'],traindf['ATM_CW_Amt_prev6'],traindf['ATM_CW_Cnt_prev6'],traindf['BRN_CW_Amt_prev6']
del traindf['BRN_CW_Cnt_prev6'],traindf['BRN_CASH_Dep_Amt_prev6'],traindf['BRN_CASH_Dep_Cnt_prev6'],traindf['CNR_prev6']
del traindf['BAL_prev6'],traindf['EOP_prev6'],traindf['CR_AMB_Prev6']

In [21]:
traindf.shape

(300000, 250)

In [22]:
# New Loan Features
traindf['PREUNSECLOSLN'] = (traindf.fillna(0)['AGRI_PREM_CLOSED_PREVQ1']+traindf.fillna(0)['CC_PREM_CLOSED_PREVQ1']+
                           traindf.fillna(0)['EDU_PREM_CLOSED_PREVQ1']+traindf.fillna(0)['PL_PREM_CLOSED_PREVQ1']+
                        traindf.fillna(0)['BL_PREM_CLOSED_PREVQ1']+traindf.fillna(0)['OTHER_LOANS_PREM_CLOSED_PREVQ1'])
traindf['PRESECLOSLN'] = (traindf.fillna(0)['AL_CNC_PREM_CLOSED_PREVQ1']+traindf.fillna(0)['AL_PREM_CLOSED_PREVQ1']+
                       traindf.fillna(0)['CE_PREM_CLOSED_PREVQ1']+traindf.fillna(0)['CV_PREM_CLOSED_PREVQ1']+
                      traindf.fillna(0)['TL_PREM_CLOSED_PREVQ1']+traindf.fillna(0)['TWL_PREM_CLOSED_PREVQ1'])
traindf['PREINVCLOS'] = (traindf.fillna(0)['RD_PREM_CLOSED_PREVQ1']+traindf.fillna(0)['FD_PREM_CLOSED_PREVQ1'])
traindf['UNSECLOSLN'] = (traindf.fillna(0)['AGRI_Closed_PrevQ1']+traindf.fillna(0)['CC_CLOSED_PREVQ1']+
                         traindf.fillna(0)['EDU_Closed_PrevQ1']+traindf.fillna(0)['OTHER_LOANS_Closed_PrevQ1']+
                         traindf.fillna(0)['PL_Closed_PrevQ1']+traindf.fillna(0)['BL_Closed_PrevQ1'])
traindf['SECLOSLN'] = (traindf.fillna(0)['AL_CNC_Closed_PrevQ1']+traindf.fillna(0)['AL_Closed_PrevQ1']+
                         traindf.fillna(0)['CE_Closed_PrevQ1']+traindf.fillna(0)['CV_Closed_PrevQ1']+
                         traindf.fillna(0)['GL_Closed_PrevQ1']+traindf.fillna(0)['TL_Closed_PrevQ1']+
                           traindf.fillna(0)['TWL_Closed_PrevQ1'])
traindf['INVCLOS'] = (traindf.fillna(0)['RD_CLOSED_PREVQ1']+traindf.fillna(0)['FD_CLOSED_PREVQ1'])
traindf['INCCLOSLY'] = (traindf.fillna(0)['DEMAT_CLOSED_PREV1YR']+traindf.fillna(0)['SEC_ACC_CLOSED_PREV1YR'])

In [23]:
del traindf['AGRI_PREM_CLOSED_PREVQ1'],traindf['AL_CNC_PREM_CLOSED_PREVQ1'],traindf['AL_PREM_CLOSED_PREVQ1']
del traindf['BL_PREM_CLOSED_PREVQ1'],traindf['CC_PREM_CLOSED_PREVQ1'],traindf['CE_PREM_CLOSED_PREVQ1']
del traindf['CV_PREM_CLOSED_PREVQ1'],traindf['EDU_PREM_CLOSED_PREVQ1'],traindf['OTHER_LOANS_PREM_CLOSED_PREVQ1']
del traindf['PL_PREM_CLOSED_PREVQ1'],traindf['RD_PREM_CLOSED_PREVQ1'],traindf['FD_PREM_CLOSED_PREVQ1']
del traindf['TL_PREM_CLOSED_PREVQ1'],traindf['TWL_PREM_CLOSED_PREVQ1'],traindf['AGRI_Closed_PrevQ1']
del traindf['AL_CNC_Closed_PrevQ1'],traindf['AL_Closed_PrevQ1'],traindf['BL_Closed_PrevQ1'],traindf['CC_CLOSED_PREVQ1']
del traindf['CE_Closed_PrevQ1'],traindf['CV_Closed_PrevQ1'],traindf['EDU_Closed_PrevQ1'],traindf['GL_Closed_PrevQ1']
del traindf['OTHER_LOANS_Closed_PrevQ1'],traindf['PL_Closed_PrevQ1'],traindf['RD_CLOSED_PREVQ1'],traindf['FD_CLOSED_PREVQ1']
del traindf['TL_Closed_PrevQ1'],traindf['TWL_Closed_PrevQ1'],traindf['SEC_ACC_CLOSED_PREV1YR'],traindf['DEMAT_CLOSED_PREV1YR']

In [24]:
traindf.shape

(300000, 226)

In [25]:
# Active Loan and Investments
traindf['LIVSECLN'] = (traindf['AL_CNC_TAG_LIVE']+traindf['AL_TAG_LIVE']+traindf['CE_TAG_LIVE']+
                      traindf['CV_TAG_LIVE']+traindf['GL_TAG_LIVE']+traindf['HL_TAG_LIVE']+
                      traindf['TL_TAG_LIVE']+traindf['TWL_TAG_LIVE']+traindf['lap_tag_live']+
                      traindf['LAS_TAG_LIVE'])
traindf['LIVUSECLN'] = (traindf['AGRI_TAG_LIVE']+traindf['BL_TAG_LIVE']+traindf['CC_TAG_LIVE']+
                      traindf['EDU_TAG_LIVE']+traindf['OTHER_LOANS_TAG_LIVE']+traindf['PL_TAG_LIVE'])
traindf['LIVINVST'] = (traindf['DEMAT_TAG_LIVE']+traindf['SEC_ACC_TAG_LIVE']+traindf['MF_TAG_LIVE']+
                       traindf['INS_TAG_LIVE']+traindf['RD_TAG_LIVE']+traindf['FD_TAG_LIVE'])

In [26]:
del traindf['AGRI_TAG_LIVE'],traindf['AL_CNC_TAG_LIVE'],traindf['AL_TAG_LIVE'],traindf['CV_TAG_LIVE']
del traindf['BL_TAG_LIVE'],traindf['CC_TAG_LIVE'],traindf['CE_TAG_LIVE'],traindf['DEMAT_TAG_LIVE']
del traindf['EDU_TAG_LIVE'],traindf['GL_TAG_LIVE'],traindf['HL_TAG_LIVE'],traindf['SEC_ACC_TAG_LIVE']
del traindf['INS_TAG_LIVE'],traindf['LAS_TAG_LIVE'],traindf['MF_TAG_LIVE'],traindf['OTHER_LOANS_TAG_LIVE']
del traindf['PL_TAG_LIVE'],traindf['RD_TAG_LIVE'],traindf['FD_TAG_LIVE'],traindf['TL_TAG_LIVE']
del traindf['TWL_TAG_LIVE'],traindf['lap_tag_live']

In [27]:
traindf.shape

(300000, 207)

In [28]:
# Fill missing values
traindf['FRX_PrevQ1']=traindf.fillna(0)['FRX_PrevQ1']
traindf['Billpay_Active_PrevQ1']=traindf.fillna(0)['Billpay_Active_PrevQ1']
traindf['Billpay_Reg_ason_Prev1']=traindf.fillna(0)['Billpay_Reg_ason_Prev1']
traindf['NO_OF_FD_BOOK_PrevQ1']=traindf.fillna(0)['NO_OF_FD_BOOK_PrevQ1']
traindf['Billpay_Active_PrevQ1']=traindf.fillna(0)['Billpay_Active_PrevQ1']
traindf['Billpay_Reg_ason_Prev1']=traindf.fillna(0)['Billpay_Reg_ason_Prev1']

In [29]:
# Total Invest value Q1 and Q2
traindf['TotInvValQ1'] = (traindf.fillna(0)['FD_AMOUNT_BOOK_PrevQ1']+traindf.fillna(0)['RD_AMOUNT_BOOK_PrevQ1']+
                          traindf.fillna(0)['Total_Invest_in_MF_PrevQ1']+traindf.fillna(0)['Dmat_Investing_PrevQ1'])
traindf['TotInvValQ2'] = (traindf.fillna(0)['FD_AMOUNT_BOOK_PrevQ2']+traindf.fillna(0)['RD_AMOUNT_BOOK_PrevQ2']+
                          traindf.fillna(0)['Total_Invest_in_MF_PrevQ2']+traindf.fillna(0)['Dmat_Investing_PrevQ2'])
traindf['TotInvCntQ1'] = (traindf.fillna(0)['NO_OF_FD_BOOK_PrevQ1']+traindf.fillna(0)['NO_OF_RD_BOOK_PrevQ1']+
                          traindf.fillna(0)['count_No_of_MF_PrevQ1'])
traindf['TotInvCntQ2'] = (traindf.fillna(0)['NO_OF_FD_BOOK_PrevQ2']+traindf.fillna(0)['NO_OF_RD_BOOK_PrevQ2']+
                          traindf.fillna(0)['count_No_of_MF_PrevQ2'])

In [30]:
del traindf['FD_AMOUNT_BOOK_PrevQ1'],traindf['RD_AMOUNT_BOOK_PrevQ1'],traindf['Total_Invest_in_MF_PrevQ1'],traindf['Dmat_Investing_PrevQ1']
del traindf['FD_AMOUNT_BOOK_PrevQ2'],traindf['RD_AMOUNT_BOOK_PrevQ2'],traindf['Total_Invest_in_MF_PrevQ2'],traindf['Dmat_Investing_PrevQ2']
del traindf['NO_OF_FD_BOOK_PrevQ1'],traindf['NO_OF_FD_BOOK_PrevQ2'],traindf['NO_OF_RD_BOOK_PrevQ1'],traindf['NO_OF_RD_BOOK_PrevQ2']
del traindf['count_No_of_MF_PrevQ1'],traindf['count_No_of_MF_PrevQ2']

In [31]:
traindf.shape

(300000, 197)

In [32]:
# replace 0 for missing NaN
traindf['NO_OF_COMPLAINTS']=traindf.fillna(0)['NO_OF_COMPLAINTS']

In [33]:
#Req_Logged_PrevQ1	Req_Resolved_PrevQ1	Query_Logged_PrevQ1	Query_Resolved_PrevQ1	Complaint_Logged_PrevQ1	Complaint_Resolved_PrevQ1
#print (traindf.Complaint_Logged_PrevQ1.value_counts(normalize=True,sort=False))
#print ('Missing % is',(traindf.Complaint_Logged_PrevQ1.isnull().sum()/len(traindf)))

In [34]:
# Replace missing value for Qury/Requests
traindf['Req_Resolved_PrevQ1'] = traindf['Req_Resolved_PrevQ1'].fillna(0.0)
traindf['Query_Resolved_PrevQ1'] = traindf['Query_Resolved_PrevQ1'].fillna(0.0)
traindf['Complaint_Resolved_PrevQ1'] = traindf['Complaint_Resolved_PrevQ1'].fillna(0.0)
traindf['Req_Logged_PrevQ1'] = traindf['Req_Logged_PrevQ1'].fillna(0.0)
traindf['Query_Logged_PrevQ1'] = traindf['Query_Logged_PrevQ1'].fillna(0.0)
traindf['Complaint_Logged_PrevQ1'] = traindf['Complaint_Logged_PrevQ1'].fillna(0.0)

In [35]:
#Replace the symbol > with 1.0
traindf['Query_Resolved_PrevQ1']=traindf['Query_Resolved_PrevQ1'].replace('>', 1.0)
traindf['Req_Resolved_PrevQ1']=traindf['Req_Resolved_PrevQ1'].replace('>', 1.0)
traindf['Complaint_Resolved_PrevQ1']=traindf['Complaint_Resolved_PrevQ1'].replace('>', 1.0)

In [36]:
# Conv the Query/Request as String
traindf['Req_Resolved_PrevQ1']=traindf['Req_Resolved_PrevQ1'].astype(float)
traindf['Query_Resolved_PrevQ1']=traindf['Query_Resolved_PrevQ1'].astype(float)
traindf['Complaint_Resolved_PrevQ1']=traindf['Complaint_Resolved_PrevQ1'].astype(float)
traindf['Req_Logged_PrevQ1']=traindf['Req_Resolved_PrevQ1'].astype(float)
traindf['Query_Logged_PrevQ1']=traindf['Query_Resolved_PrevQ1'].astype(float)
traindf['Complaint_Logged_PrevQ1']=traindf['Complaint_Resolved_PrevQ1'].astype(float)

In [37]:
# Create a new variable to gather happiness index
traindf['Reqresperc']=traindf.fillna(0.0)['Req_Resolved_PrevQ1']/traindf.fillna(0.0)['Req_Logged_PrevQ1']
traindf['Qryresperc']=traindf.fillna(0.0)['Query_Resolved_PrevQ1']/traindf.fillna(0.0)['Query_Logged_PrevQ1']
traindf['Cmpresperc']=traindf.fillna(0.0)['Complaint_Resolved_PrevQ1']/traindf.fillna(0.0)['Complaint_Logged_PrevQ1']

In [38]:
# replace 0 for Nan for new variables
#traindf['Reqresperc']=traindf.fillna(0)['Reqresperc']
#traindf['Qryresperc']=traindf.fillna(0)['Qryresperc']
#traindf['Cmpresperc']=traindf.fillna(0)['Cmpresperc']
del traindf['Req_Resolved_PrevQ1'],traindf['Query_Resolved_PrevQ1'],traindf['Complaint_Resolved_PrevQ1'],traindf['Req_Logged_PrevQ1'],traindf['Query_Logged_PrevQ1'],traindf['Complaint_Logged_PrevQ1']
traindf.shape

(300000, 194)

In [39]:
del traindf['AGRI_DATE'],traindf['AL_CNC_DATE'],traindf['AL_DATE'],traindf['BL_DATE'],traindf['CE_DATE']
del traindf['CV_DATE'],traindf['EDU_DATE'],traindf['GL_DATE'],traindf['LAP_DATE'],traindf['LAS_DATE']
del traindf['OTHER_LOANS_DATE'],traindf['PL_DATE'],traindf['TL_DATE'],traindf['TWL_DATE']
traindf.shape

(300000, 180)

In [40]:
# Checking for the ratio of missing value
Total = traindf.isnull().sum().sort_values(ascending=False)
percent = ((traindf.isnull().sum()/traindf.isnull().count())*100).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :percent, 'Missing Total':Total})
missing_data.head()

,Missing Ratio,Missing Total
POSAvgCRDiff4,100.0,300000
ATMAvgCRDiff1,100.0,300000
MBAvgCRDiff3,100.0,300000
POSAvgCRDiff1,100.0,300000
MBAvgCRDiff1,100.0,300000


In [41]:
print (len(missing_data))
print (len(missing_data[missing_data["Missing Total"] <= 100000] ))
print (len(missing_data[missing_data["Missing Total"] >= 100000] ))
print (len(missing_data[missing_data["Missing Total"] >= 250000] ))

180
82
98
57


In [42]:
cols_consider = ['UCIC_ID','Percent_Change_in_Credits','Recency_of_CR_TXN','Recency_of_DR_TXN','Recency_of_BRANCH_TXN','Recency_of_Activity','ATMWithDiff1','gender_bin','BALDiff1','TotInvCntQ2','RBI_Class_Audit','I_AQB_PrevQ1','I_AQB_PrevQ2','I_CR_AQB_PrevQ1','I_CR_AQB_PrevQ2','I_CNR_PrevQ1','I_CNR_PrevQ2','I_NRV_PrevQ1','age','Billpay_Reg_ason_Prev1_N','FRX_PrevQ1_N','FRX_PrevQ1','NO_OF_Accs','HNW_CATEGORY','vintage','EMAIL_UNSUBSCRIBE','OCCUP_ALL_NEW','dependents','FINAL_WORTH_prev1','ENGAGEMENT_TAG_prev1','EFT_SELF_TRANSFER_PrevQ1','Charges_cnt_PrevQ1_N','Billpay_Active_PrevQ1','Billpay_Reg_ason_Prev1','Charges_PrevQ1','NO_OF_COMPLAINTS','CASH_WD_AMT_Last6','CASH_WD_CNT_Last6','Billpay_Active_PrevQ1_N','CR_AMB_Drop_Build_1','I_NRV_PrevQ2','TotInvCntQ1','CR_AMB_Drop_Build_2','BALDiff5','ATMWithDiff5','CRAMBDiff4','EOPDiff4','BALDiff4','CNTDiff4','ATMWithDiff4','CRAMBDiff3','EOPDiff3','BALDiff3','CNTDiff3','ATMWithDiff3','CRAMBDiff2','EOPDiff2','BALDiff2','CNTDiff2','ATMWithDiff2','CRAMBDiff1','CNTDiff5','CNTDiff1','CR_AMB_Drop_Build_3','EOPDiff5','CR_AMB_Drop_Build_4','CR_AMB_Drop_Build_5','EOPDiff1','TotInvValQ2','TotInvValQ1','LIVINVST','Responders','LIVUSECLN','LIVSECLN','INCCLOSLY','INVCLOS','SECLOSLN','UNSECLOSLN','PREINVCLOS','PRESECLOSLN','PREUNSECLOSLN','CRAMBDiff5']

In [43]:
traindf = traindf[cols_consider]
traindf.shape

(300000, 82)

In [44]:
# Idnetify the non numric cols for encoding
nonnumcols = traindf.dtypes[traindf.dtypes == "object"].index
print (len(nonnumcols))
print (nonnumcols)

12
Index(['gender_bin', 'RBI_Class_Audit', 'Billpay_Reg_ason_Prev1_N',
       'FRX_PrevQ1_N', 'HNW_CATEGORY', 'EMAIL_UNSUBSCRIBE', 'OCCUP_ALL_NEW',
       'FINAL_WORTH_prev1', 'ENGAGEMENT_TAG_prev1', 'EFT_SELF_TRANSFER_PrevQ1',
       'Charges_cnt_PrevQ1_N', 'Billpay_Active_PrevQ1_N'],
      dtype='object')


In [45]:
traindf = pd.get_dummies(traindf, columns=['gender_bin', 'RBI_Class_Audit', 'Billpay_Reg_ason_Prev1_N',
       'FRX_PrevQ1_N', 'HNW_CATEGORY', 'EMAIL_UNSUBSCRIBE', 'OCCUP_ALL_NEW',
       'FINAL_WORTH_prev1', 'ENGAGEMENT_TAG_prev1', 'EFT_SELF_TRANSFER_PrevQ1',
       'Charges_cnt_PrevQ1_N', 'Billpay_Active_PrevQ1_N'], drop_first=True)
print ('After One hot encoding',traindf.shape)
#,'AGEBKT'

After One hot encoding (300000, 97)


In [46]:
# Import the test data
#testdf = pd.read_csv('/Users/Vishy/Files/AVDatahack/Churn/RawData/test.csv')
testdf = pd.read_csv('/data/test.csv')
print (testdf.shape)
testdf.head()

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (294,299,300,302,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(200000, 376)


,UCIC_ID,NO_OF_Accs,HNW_CATEGORY,vintage,EMAIL_UNSUBSCRIBE,OCCUP_ALL_NEW,city,dependents,zip,FINAL_WORTH_prev1,ENGAGEMENT_TAG_prev1,C_prev1,D_prev1,ATM_C_prev1,ATM_D_prev1,BRANCH_C_prev1,BRANCH_D_prev1,IB_C_prev1,IB_D_prev1,MB_C_prev1,MB_D_prev1,POS_C_prev1,POS_D_prev1,count_C_prev1,count_D_prev1,COUNT_ATM_C_prev1,COUNT_ATM_D_prev1,COUNT_BRANCH_C_prev1,COUNT_BRANCH_D_prev1,COUNT_IB_C_prev1,COUNT_IB_D_prev1,COUNT_MB_C_prev1,COUNT_MB_D_prev1,COUNT_POS_C_prev1,COUNT_POS_D_prev1,custinit_CR_amt_prev1,custinit_DR_amt_prev1,custinit_CR_cnt_prev1,custinit_DR_cnt_prev1,ATM_amt_prev1,ATM_CW_Amt_prev1,ATM_CW_Cnt_prev1,BRN_CW_Amt_prev1,BRN_CW_Cnt_prev1,BRN_CASH_Dep_Amt_prev1,BRN_CASH_Dep_Cnt_prev1,CNR_prev1,BAL_prev1,EOP_prev1,CR_AMB_Prev1,C_prev2,D_prev2,ATM_C_prev2,ATM_D_prev2,BRANCH_C_prev2,BRANCH_D_prev2,IB_C_prev2,IB_D_prev2,MB_C_prev2,MB_D_prev2,POS_C_prev2,POS_D_prev2,count_C_prev2,count_D_prev2,COUNT_ATM_C_prev2,COUNT_ATM_D_prev2,COUNT_BRANCH_C_prev2,COUNT_BRANCH_D_prev2,COUNT_IB_C_prev2,COUNT_IB_D_prev2,COUNT_MB_C_prev2,COUNT_MB_D_prev2,COUNT_POS_C_prev2,COUNT_POS_D_prev2,custinit_CR_amt_prev2,custinit_DR_amt_prev2,custinit_CR_cnt_prev2,custinit_DR_cnt_prev2,ATM_amt_prev2,ATM_CW_Amt_prev2,ATM_CW_Cnt_prev2,BRN_CW_Amt_prev2,BRN_CW_Cnt_prev2,BRN_CASH_Dep_Amt_prev2,BRN_CASH_Dep_Cnt_prev2,CNR_prev2,BAL_prev2,EOP_prev2,CR_AMB_Prev2,C_prev3,D_prev3,ATM_C_prev3,ATM_D_prev3,BRANCH_C_prev3,BRANCH_D_prev3,IB_C_prev3,IB_D_prev3,MB_C_prev3,MB_D_prev3,POS_C_prev3,POS_D_prev3,count_C_prev3,count_D_prev3,COUNT_ATM_C_prev3,COUNT_ATM_D_prev3,COUNT_BRANCH_C_prev3,COUNT_BRANCH_D_prev3,COUNT_IB_C_prev3,COUNT_IB_D_prev3,COUNT_MB_C_prev3,COUNT_MB_D_prev3,COUNT_POS_C_prev3,COUNT_POS_D_prev3,custinit_CR_amt_prev3,custinit_DR_amt_prev3,custinit_CR_cnt_prev3,custinit_DR_cnt_prev3,ATM_amt_prev3,ATM_CW_Amt_prev3,ATM_CW_Cnt_prev3,BRN_CW_Amt_prev3,BRN_CW_Cnt_prev3,BRN_CASH_Dep_Amt_prev3,BRN_CASH_Dep_Cnt_prev3,CNR_prev3,BAL_prev3,EOP_prev3,CR_AMB_Prev3,C_prev4,D_prev4,ATM_C_prev4,ATM_D_prev4,BRANCH_C_prev4,BRANCH_D_prev4,IB_C_prev4,IB_D_prev4,MB_C_prev4,MB_D_prev4,POS_C_prev4,POS_D_prev4,count_C_prev4,count_D_prev4,COUNT_ATM_C_prev4,COUNT_ATM_D_prev4,COUNT_BRANCH_C_prev4,COUNT_BRANCH_D_prev4,COUNT_IB_C_prev4,COUNT_IB_D_prev4,COUNT_MB_C_prev4,COUNT_MB_D_prev4,COUNT_POS_C_prev4,COUNT_POS_D_prev4,custinit_CR_amt_prev4,custinit_DR_amt_prev4,custinit_CR_cnt_prev4,custinit_DR_cnt_prev4,ATM_amt_prev4,ATM_CW_Amt_prev4,ATM_CW_Cnt_prev4,BRN_CW_Amt_prev4,BRN_CW_Cnt_prev4,BRN_CASH_Dep_Amt_prev4,BRN_CASH_Dep_Cnt_prev4,CNR_prev4,BAL_prev4,EOP_prev4,CR_AMB_Prev4,C_prev5,D_prev5,ATM_C_prev5,ATM_D_prev5,BRANCH_C_prev5,BRANCH_D_prev5,IB_C_prev5,IB_D_prev5,MB_C_prev5,MB_D_prev5,POS_C_prev5,POS_D_prev5,count_C_prev5,count_D_prev5,COUNT_ATM_C_prev5,COUNT_ATM_D_prev5,COUNT_BRANCH_C_prev5,COUNT_BRANCH_D_prev5,COUNT_IB_C_prev5,COUNT_IB_D_prev5,COUNT_MB_C_prev5,COUNT_MB_D_prev5,COUNT_POS_C_prev5,COUNT_POS_D_prev5,custinit_CR_amt_prev5,custinit_DR_amt_prev5,custinit_CR_cnt_prev5,custinit_DR_cnt_prev5,ATM_amt_prev5,ATM_CW_Amt_prev5,ATM_CW_Cnt_prev5,BRN_CW_Amt_prev5,BRN_CW_Cnt_prev5,BRN_CASH_Dep_Amt_prev5,BRN_CASH_Dep_Cnt_prev5,CNR_prev5,BAL_prev5,EOP_prev5,CR_AMB_Prev5,C_prev6,D_prev6,ATM_C_prev6,ATM_D_prev6,BRANCH_C_prev6,BRANCH_D_prev6,IB_C_prev6,IB_D_prev6,MB_C_prev6,MB_D_prev6,POS_C_prev6,POS_D_prev6,count_C_prev6,count_D_prev6,COUNT_ATM_C_prev6,COUNT_ATM_D_prev6,COUNT_BRANCH_C_prev6,COUNT_BRANCH_D_prev6,COUNT_IB_C_prev6,COUNT_IB_D_prev6,COUNT_MB_C_prev6,COUNT_MB_D_prev6,COUNT_POS_C_prev6,COUNT_POS_D_prev6,custinit_CR_amt_prev6,custinit_DR_amt_prev6,custinit_CR_cnt_prev6,custinit_DR_cnt_prev6,ATM_amt_prev6,ATM_CW_Amt_prev6,ATM_CW_Cnt_prev6,BRN_CW_Amt_prev6,BRN_CW_Cnt_prev6,BRN_CASH_Dep_Amt_prev6,BRN_CASH_Dep_Cnt_prev6,CNR_prev6,BAL_prev6,EOP_prev6,CR_AMB_Prev6,FRX_PrevQ1,EFT_SELF_TRANSFER_PrevQ1,Billpay_Active_PrevQ1,Billpay_Reg_ason_Prev1,FD_AMOUNT_BOOK_PrevQ1,FD_AMOUNT_BOOK_PrevQ2,NO_OF_FD_BOOK_PrevQ1,NO_OF_FD_BOOK_PrevQ2,NO_OF_RD_BOOK_PrevQ1,NO_OF_RD_BOOK_PrevQ2,RD_AMOUNT_BOOK_Pr

In [47]:
# Filling in the missing value
testdf['OCCUP_ALL_NEW'] = testdf['OCCUP_ALL_NEW'].fillna(pd.Series(np.random.choice(['SELF_EMPLOYED','SALARIED','HOUSEWIFE'], p=[0.50,0.30,0.20], size=len(testdf))))
testdf['dependents'] = testdf['dependents'].fillna(pd.Series(np.random.choice([0,2,1], p=[0.85,0.10,0.05], size=len(testdf))))
testdf['FINAL_WORTH_prev1'] = testdf['FINAL_WORTH_prev1'].fillna(pd.Series(np.random.choice(['HIGH','MEDIUM'], p=[0.75,0.25], size=len(testdf))))
testdf['ENGAGEMENT_TAG_prev1'] = testdf['ENGAGEMENT_TAG_prev1'].fillna(pd.Series(np.random.choice(['LOW','HIGH','MEDIUM'], p=[0.50,0.30,0.20], size=len(testdf))))

In [48]:
testdf['LAS_TAG_LIVE']=0

In [49]:
# Only Y columns
testdf['EMAIL_UNSUBSCRIBE'] = np.where(testdf['EMAIL_UNSUBSCRIBE']=='Y', 'Y', 'N')
testdf['AGRI_TAG_LIVE'] = np.where(testdf['AGRI_TAG_LIVE']=='Y', 1, 0)
testdf['AL_CNC_TAG_LIVE'] = np.where(testdf['AL_CNC_TAG_LIVE']=='Y', 1, 0)
testdf['AL_TAG_LIVE'] = np.where(testdf['AL_TAG_LIVE']=='Y', 1, 0)
testdf['BL_TAG_LIVE'] = np.where(testdf['BL_TAG_LIVE']=='Y', 1, 0)
testdf['CC_TAG_LIVE'] = np.where(testdf['CC_TAG_LIVE']=='Y', 1, 0)
testdf['CE_TAG_LIVE'] = np.where(testdf['CE_TAG_LIVE']=='Y', 1, 0)
testdf['CV_TAG_LIVE'] = np.where(testdf['CV_TAG_LIVE']=='Y', 1, 0)
testdf['DEMAT_TAG_LIVE'] = np.where(testdf['DEMAT_TAG_LIVE']=='Y', 1, 0)
testdf['EDU_TAG_LIVE'] = np.where(testdf['EDU_TAG_LIVE']=='Y', 1, 0)
testdf['GL_TAG_LIVE'] = np.where(testdf['GL_TAG_LIVE']=='Y', 1, 0)
testdf['HL_TAG_LIVE'] = np.where(testdf['HL_TAG_LIVE']=='Y', 1, 0)
testdf['SEC_ACC_TAG_LIVE'] = np.where(testdf['SEC_ACC_TAG_LIVE']=='Y', 1, 0)
testdf['INS_TAG_LIVE'] = np.where(testdf['INS_TAG_LIVE']=='Y', 1, 0)
testdf['MF_TAG_LIVE'] = np.where(testdf['MF_TAG_LIVE']=='Y', 1, 0)
testdf['OTHER_LOANS_TAG_LIVE'] = np.where(testdf['OTHER_LOANS_TAG_LIVE']=='Y', 1, 0)
testdf['PL_TAG_LIVE'] = np.where(testdf['PL_TAG_LIVE']=='Y', 1, 0)
testdf['RD_TAG_LIVE'] = np.where(testdf['RD_TAG_LIVE']=='Y', 1, 0)
testdf['FD_TAG_LIVE'] = np.where(testdf['FD_TAG_LIVE']=='Y', 1, 0)
testdf['TL_TAG_LIVE'] = np.where(testdf['TL_TAG_LIVE']=='Y', 1, 0)
testdf['TWL_TAG_LIVE'] = np.where(testdf['TWL_TAG_LIVE']=='Y', 1, 0)
testdf['lap_tag_live'] = np.where(testdf['lap_tag_live']=='Y', 1, 0)
testdf['EFT_SELF_TRANSFER_PrevQ1'] = np.where(testdf['EFT_SELF_TRANSFER_PrevQ1']=='Y','Y','N')

In [50]:
del testdf['city'], testdf['zip'], testdf['brn_code']

In [51]:
# New variables
#Diff1
testdf['TAvgCRDiff1'] = ((testdf.fillna(0)['C_prev1']/testdf.fillna(0)['count_C_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['C_prev2']/testdf.fillna(0)['count_C_prev2']).replace(np.inf, 0))
testdf['TAvgDRDiff1'] = ((testdf.fillna(0)['D_prev1']/testdf.fillna(0)['count_D_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['D_prev2']/testdf.fillna(0)['count_C_prev2']).replace(np.inf, 0))
testdf['ATMAvgCRDiff1'] = ((testdf.fillna(0)['ATM_C_prev1']/testdf.fillna(0)['COUNT_ATM_C_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['ATM_C_prev2']/testdf.fillna(0)['COUNT_ATM_C_prev2']).replace(np.inf, 0))
testdf['ATMAvgDRDiff1'] = ((testdf.fillna(0)['ATM_D_prev1']/testdf.fillna(0)['COUNT_ATM_D_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['ATM_D_prev2']/testdf.fillna(0)['COUNT_ATM_D_prev2']).replace(np.inf, 0))
testdf['BRAvgCRDiff1'] = ((testdf.fillna(0)['BRANCH_C_prev1']/testdf.fillna(0)['COUNT_BRANCH_C_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['BRANCH_C_prev2']/testdf.fillna(0)['COUNT_BRANCH_C_prev2']).replace(np.inf, 0))
testdf['BRAvgDRDiff1'] = ((testdf.fillna(0)['BRANCH_D_prev1']/testdf.fillna(0)['COUNT_BRANCH_D_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['BRANCH_D_prev2']/testdf.fillna(0)['COUNT_BRANCH_D_prev2']).replace(np.inf, 0))
testdf['IBAvgCRDiff1'] = ((testdf.fillna(0)['IB_C_prev1']/testdf.fillna(0)['COUNT_IB_C_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['IB_C_prev2']/testdf.fillna(0)['COUNT_IB_C_prev2']).replace(np.inf, 0))
testdf['IBAvgDRDiff1'] = ((testdf.fillna(0)['IB_D_prev1']/testdf.fillna(0)['COUNT_IB_D_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['IB_D_prev2']/testdf.fillna(0)['COUNT_IB_D_prev2']).replace(np.inf, 0))
testdf['MBAvgCRDiff1'] = ((testdf.fillna(0)['MB_C_prev1']/testdf.fillna(0)['COUNT_MB_C_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['MB_C_prev2']/testdf.fillna(0)['COUNT_MB_C_prev2']).replace(np.inf, 0))
testdf['MBAvgDRDiff1'] = ((testdf.fillna(0)['MB_D_prev1']/testdf.fillna(0)['COUNT_MB_D_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['MB_D_prev2']/testdf.fillna(0)['COUNT_MB_D_prev2']).replace(np.inf, 0))
testdf['POSAvgCRDiff1'] = ((testdf.fillna(0)['POS_C_prev1']/testdf.fillna(0)['COUNT_POS_C_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['POS_C_prev2']/testdf.fillna(0)['COUNT_POS_C_prev2']).replace(np.inf, 0))
testdf['POSAvgDRDiff1'] = ((testdf.fillna(0)['POS_D_prev1']/testdf.fillna(0)['COUNT_POS_D_prev1']).replace(np.inf, 0)) - ((testdf.fillna(0)['POS_D_prev2']/testdf.fillna(0)['COUNT_POS_D_prev2']).replace(np.inf, 0))
#Diff2
testdf['TAvgCRDiff2'] = ((testdf.fillna(0)['C_prev2']/testdf.fillna(0)['count_C_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['C_prev3']/testdf.fillna(0)['count_C_prev3']).replace(np.inf, 0))
testdf['TAvgDRDiff2'] = ((testdf.fillna(0)['D_prev2']/testdf.fillna(0)['count_D_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['D_prev3']/testdf.fillna(0)['count_C_prev3']).replace(np.inf, 0))
testdf['ATMAvgCRDiff2'] = ((testdf.fillna(0)['ATM_C_prev2']/testdf.fillna(0)['COUNT_ATM_C_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['ATM_C_prev3']/testdf.fillna(0)['COUNT_ATM_C_prev3']).replace(np.inf, 0))
testdf['ATMAvgDRDiff2'] = ((testdf.fillna(0)['ATM_D_prev2']/testdf.fillna(0)['COUNT_ATM_D_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['ATM_D_prev3']/testdf.fillna(0)['COUNT_ATM_D_prev3']).replace(np.inf, 0))
testdf['BRAvgCRDiff2'] = ((testdf.fillna(0)['BRANCH_C_prev2']/testdf.fillna(0)['COUNT_BRANCH_C_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['BRANCH_C_prev3']/testdf.fillna(0)['COUNT_BRANCH_C_prev3']).replace(np.inf, 0))
testdf['BRAvgDRDiff2'] = ((testdf.fillna(0)['BRANCH_D_prev2']/testdf.fillna(0)['COUNT_BRANCH_D_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['BRANCH_D_prev3']/testdf.fillna(0)['COUNT_BRANCH_D_prev3']).replace(np.inf, 0))
testdf['IBAvgCRDiff2'] = ((testdf.fillna(0)['IB_C_prev2']/testdf.fillna(0)['COUNT_IB_C_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['IB_C_prev3']/testdf.fillna(0)['COUNT_IB_C_prev3']).replace(np.inf, 0))
testdf['IBAvgDRDiff2'] = ((testdf.fillna(0)['IB_D_prev2']/testdf.fillna(0)['COUNT_IB_D_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['IB_D_prev3']/testdf.fillna(0)['COUNT_IB_D_prev3']).replace(np.inf, 0))
testdf['MBAvgCRDiff2'] = ((testdf.fillna(0)['MB_C_prev2']/testdf.fillna(0)['COUNT_MB_C_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['MB_C_prev3']/testdf.fillna(0)['COUNT_MB_C_prev3']).replace(np.inf, 0))
testdf['MBAvgDRDiff2'] = ((testdf.fillna(0)['MB_D_prev2']/testdf.fillna(0)['COUNT_MB_D_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['MB_D_prev3']/testdf.fillna(0)['COUNT_MB_D_prev3']).replace(np.inf, 0))
testdf['POSAvgCRDiff2'] = ((testdf.fillna(0)['POS_C_prev2']/testdf.fillna(0)['COUNT_POS_C_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['POS_C_prev3']/testdf.fillna(0)['COUNT_POS_C_prev3']).replace(np.inf, 0))
testdf['POSAvgDRDiff2'] = ((testdf.fillna(0)['POS_D_prev2']/testdf.fillna(0)['COUNT_POS_D_prev2']).replace(np.inf, 0)) - ((testdf.fillna(0)['POS_D_prev3']/testdf.fillna(0)['COUNT_POS_D_prev3']).replace(np.inf, 0))
#Diff3
testdf['TAvgCRDiff3'] = ((testdf.fillna(0)['C_prev3']/testdf.fillna(0)['count_C_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['C_prev4']/testdf.fillna(0)['count_C_prev4']).replace(np.inf, 0))
testdf['TAvgDRDiff3'] = ((testdf.fillna(0)['D_prev3']/testdf.fillna(0)['count_D_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['D_prev4']/testdf.fillna(0)['count_C_prev4']).replace(np.inf, 0))
testdf['ATMAvgCRDiff3'] = ((testdf.fillna(0)['ATM_C_prev3']/testdf.fillna(0)['COUNT_ATM_C_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['ATM_C_prev4']/testdf.fillna(0)['COUNT_ATM_C_prev4']).replace(np.inf, 0))
testdf['ATMAvgDRDiff3'] = ((testdf.fillna(0)['ATM_D_prev3']/testdf.fillna(0)['COUNT_ATM_D_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['ATM_D_prev4']/testdf.fillna(0)['COUNT_ATM_D_prev4']).replace(np.inf, 0))
testdf['BRAvgCRDiff3'] = ((testdf.fillna(0)['BRANCH_C_prev3']/testdf.fillna(0)['COUNT_BRANCH_C_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['BRANCH_C_prev4']/testdf.fillna(0)['COUNT_BRANCH_C_prev4']).replace(np.inf, 0))
testdf['BRAvgDRDiff3'] = ((testdf.fillna(0)['BRANCH_D_prev3']/testdf.fillna(0)['COUNT_BRANCH_D_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['BRANCH_D_prev4']/testdf.fillna(0)['COUNT_BRANCH_D_prev4']).replace(np.inf, 0))
testdf['IBAvgCRDiff3'] = ((testdf.fillna(0)['IB_C_prev3']/testdf.fillna(0)['COUNT_IB_C_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['IB_C_prev4']/testdf.fillna(0)['COUNT_IB_C_prev4']).replace(np.inf, 0))
testdf['IBAvgDRDiff3'] = ((testdf.fillna(0)['IB_D_prev3']/testdf.fillna(0)['COUNT_IB_D_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['IB_D_prev4']/testdf.fillna(0)['COUNT_IB_D_prev4']).replace(np.inf, 0))
testdf['MBAvgCRDiff3'] = ((testdf.fillna(0)['MB_C_prev3']/testdf.fillna(0)['COUNT_MB_C_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['MB_C_prev4']/testdf.fillna(0)['COUNT_MB_C_prev4']).replace(np.inf, 0))
testdf['MBAvgDRDiff3'] = ((testdf.fillna(0)['MB_D_prev3']/testdf.fillna(0)['COUNT_MB_D_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['MB_D_prev4']/testdf.fillna(0)['COUNT_MB_D_prev4']).replace(np.inf, 0))
testdf['POSAvgCRDiff3'] = ((testdf.fillna(0)['POS_C_prev3']/testdf.fillna(0)['COUNT_POS_C_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['POS_C_prev4']/testdf.fillna(0)['COUNT_POS_C_prev4']).replace(np.inf, 0))
testdf['POSAvgDRDiff3'] = ((testdf.fillna(0)['POS_D_prev3']/testdf.fillna(0)['COUNT_POS_D_prev3']).replace(np.inf, 0)) - ((testdf.fillna(0)['POS_D_prev4']/testdf.fillna(0)['COUNT_POS_D_prev4']).replace(np.inf, 0))
#Diff4
testdf['TAvgCRDiff4'] = ((testdf.fillna(0)['C_prev4']/testdf.fillna(0)['count_C_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['C_prev5']/testdf.fillna(0)['count_C_prev5']).replace(np.inf, 0))
testdf['TAvgDRDiff4'] = ((testdf.fillna(0)['D_prev4']/testdf.fillna(0)['count_D_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['D_prev5']/testdf.fillna(0)['count_C_prev5']).replace(np.inf, 0))
testdf['ATMAvgCRDiff4'] = ((testdf.fillna(0)['ATM_C_prev4']/testdf.fillna(0)['COUNT_ATM_C_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['ATM_C_prev5']/testdf.fillna(0)['COUNT_ATM_C_prev5']).replace(np.inf, 0))
testdf['ATMAvgDRDiff4'] = ((testdf.fillna(0)['ATM_D_prev4']/testdf.fillna(0)['COUNT_ATM_D_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['ATM_D_prev5']/testdf.fillna(0)['COUNT_ATM_D_prev5']).replace(np.inf, 0))
testdf['BRAvgCRDiff4'] = ((testdf.fillna(0)['BRANCH_C_prev4']/testdf.fillna(0)['COUNT_BRANCH_C_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['BRANCH_C_prev5']/testdf.fillna(0)['COUNT_BRANCH_C_prev5']).replace(np.inf, 0))
testdf['BRAvgDRDiff4'] = ((testdf.fillna(0)['BRANCH_D_prev4']/testdf.fillna(0)['COUNT_BRANCH_D_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['BRANCH_D_prev5']/testdf.fillna(0)['COUNT_BRANCH_D_prev5']).replace(np.inf, 0))
testdf['IBAvgCRDiff4'] = ((testdf.fillna(0)['IB_C_prev4']/testdf.fillna(0)['COUNT_IB_C_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['IB_C_prev5']/testdf.fillna(0)['COUNT_IB_C_prev5']).replace(np.inf, 0))
testdf['IBAvgDRDiff4'] = ((testdf.fillna(0)['IB_D_prev4']/testdf.fillna(0)['COUNT_IB_D_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['IB_D_prev5']/testdf.fillna(0)['COUNT_IB_D_prev5']).replace(np.inf, 0))
testdf['MBAvgCRDiff4'] = ((testdf.fillna(0)['MB_C_prev4']/testdf.fillna(0)['COUNT_MB_C_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['MB_C_prev5']/testdf.fillna(0)['COUNT_MB_C_prev5']).replace(np.inf, 0))
testdf['MBAvgDRDiff4'] = ((testdf.fillna(0)['MB_D_prev4']/testdf.fillna(0)['COUNT_MB_D_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['MB_D_prev5']/testdf.fillna(0)['COUNT_MB_D_prev5']).replace(np.inf, 0))
testdf['POSAvgCRDiff4'] = ((testdf.fillna(0)['POS_C_prev4']/testdf.fillna(0)['COUNT_POS_C_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['POS_C_prev5']/testdf.fillna(0)['COUNT_POS_C_prev5']).replace(np.inf, 0))
testdf['POSAvgDRDiff4'] = ((testdf.fillna(0)['POS_D_prev4']/testdf.fillna(0)['COUNT_POS_D_prev4']).replace(np.inf, 0)) - ((testdf.fillna(0)['POS_D_prev5']/testdf.fillna(0)['COUNT_POS_D_prev5']).replace(np.inf, 0))
#Diff5
testdf['TAvgCRDiff5'] = ((testdf.fillna(0)['C_prev5']/testdf.fillna(0)['count_C_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['C_prev6']/testdf.fillna(0)['count_C_prev6']).replace(np.inf, 0))
testdf['TAvgDRDiff5'] = ((testdf.fillna(0)['D_prev5']/testdf.fillna(0)['count_D_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['D_prev6']/testdf.fillna(0)['count_C_prev6']).replace(np.inf, 0))
testdf['ATMAvgCRDiff5'] = ((testdf.fillna(0)['ATM_C_prev5']/testdf.fillna(0)['COUNT_ATM_C_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['ATM_C_prev6']/testdf.fillna(0)['COUNT_ATM_C_prev6']).replace(np.inf, 0))
testdf['ATMAvgDRDiff5'] = ((testdf.fillna(0)['ATM_D_prev5']/testdf.fillna(0)['COUNT_ATM_D_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['ATM_D_prev6']/testdf.fillna(0)['COUNT_ATM_D_prev6']).replace(np.inf, 0))
testdf['BRAvgCRDiff5'] = ((testdf.fillna(0)['BRANCH_C_prev5']/testdf.fillna(0)['COUNT_BRANCH_C_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['BRANCH_C_prev6']/testdf.fillna(0)['COUNT_BRANCH_C_prev6']).replace(np.inf, 0))
testdf['BRAvgDRDiff5'] = ((testdf.fillna(0)['BRANCH_D_prev5']/testdf.fillna(0)['COUNT_BRANCH_D_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['BRANCH_D_prev6']/testdf.fillna(0)['COUNT_BRANCH_D_prev6']).replace(np.inf, 0))
testdf['IBAvgCRDiff5'] = ((testdf.fillna(0)['IB_C_prev5']/testdf.fillna(0)['COUNT_IB_C_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['IB_C_prev6']/testdf.fillna(0)['COUNT_IB_C_prev6']).replace(np.inf, 0))
testdf['IBAvgDRDiff5'] = ((testdf.fillna(0)['IB_D_prev5']/testdf.fillna(0)['COUNT_IB_D_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['IB_D_prev6']/testdf.fillna(0)['COUNT_IB_D_prev6']).replace(np.inf, 0))
testdf['MBAvgCRDiff5'] = ((testdf.fillna(0)['MB_C_prev5']/testdf.fillna(0)['COUNT_MB_C_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['MB_C_prev6']/testdf.fillna(0)['COUNT_MB_C_prev6']).replace(np.inf, 0))
testdf['MBAvgDRDiff5'] = ((testdf.fillna(0)['MB_D_prev5']/testdf.fillna(0)['COUNT_MB_D_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['MB_D_prev6']/testdf.fillna(0)['COUNT_MB_D_prev6']).replace(np.inf, 0))
testdf['POSAvgCRDiff5'] = ((testdf.fillna(0)['POS_C_prev5']/testdf.fillna(0)['COUNT_POS_C_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['POS_C_prev6']/testdf.fillna(0)['COUNT_POS_C_prev6']).replace(np.inf, 0))
testdf['POSAvgDRDiff5'] = ((testdf.fillna(0)['POS_D_prev5']/testdf.fillna(0)['COUNT_POS_D_prev5']).replace(np.inf, 0)) - ((testdf.fillna(0)['POS_D_prev6']/testdf.fillna(0)['COUNT_POS_D_prev6']).replace(np.inf, 0))

In [52]:
# Delete 1
del testdf['C_prev1'],testdf['D_prev1'],testdf['ATM_C_prev1'],testdf['ATM_D_prev1'],testdf['BRANCH_C_prev1']
del testdf['BRANCH_D_prev1'],testdf['IB_C_prev1'],testdf['IB_D_prev1'],testdf['MB_C_prev1'],testdf['MB_D_prev1']
del testdf['POS_C_prev1'],testdf['POS_D_prev1'],testdf['count_C_prev1'],testdf['count_D_prev1'],testdf['COUNT_ATM_C_prev1']
del testdf['COUNT_ATM_D_prev1'],testdf['COUNT_BRANCH_C_prev1'],testdf['COUNT_BRANCH_D_prev1'],testdf['COUNT_IB_C_prev1']
del testdf['COUNT_IB_D_prev1'],testdf['COUNT_MB_C_prev1'],testdf['COUNT_MB_D_prev1'],testdf['COUNT_POS_C_prev1'],testdf['COUNT_POS_D_prev1']
# Delete 2
del testdf['C_prev2'],testdf['D_prev2'],testdf['ATM_C_prev2'],testdf['ATM_D_prev2'],testdf['BRANCH_C_prev2']
del testdf['BRANCH_D_prev2'],testdf['IB_C_prev2'],testdf['IB_D_prev2'],testdf['MB_C_prev2'],testdf['MB_D_prev2']
del testdf['POS_C_prev2'],testdf['POS_D_prev2'],testdf['count_C_prev2'],testdf['count_D_prev2'],testdf['COUNT_ATM_C_prev2']
del testdf['COUNT_ATM_D_prev2'],testdf['COUNT_BRANCH_C_prev2'],testdf['COUNT_BRANCH_D_prev2'],testdf['COUNT_IB_C_prev2']
del testdf['COUNT_IB_D_prev2'],testdf['COUNT_MB_C_prev2'],testdf['COUNT_MB_D_prev2'],testdf['COUNT_POS_C_prev2'],testdf['COUNT_POS_D_prev2']
# Delete 3
del testdf['C_prev3'],testdf['D_prev3'],testdf['ATM_C_prev3'],testdf['ATM_D_prev3'],testdf['BRANCH_C_prev3']
del testdf['BRANCH_D_prev3'],testdf['IB_C_prev3'],testdf['IB_D_prev3'],testdf['MB_C_prev3'],testdf['MB_D_prev3']
del testdf['POS_C_prev3'],testdf['POS_D_prev3'],testdf['count_C_prev3'],testdf['count_D_prev3'],testdf['COUNT_ATM_C_prev3']
del testdf['COUNT_ATM_D_prev3'],testdf['COUNT_BRANCH_C_prev3'],testdf['COUNT_BRANCH_D_prev3'],testdf['COUNT_IB_C_prev3']
del testdf['COUNT_IB_D_prev3'],testdf['COUNT_MB_C_prev3'],testdf['COUNT_MB_D_prev3'],testdf['COUNT_POS_C_prev3'],testdf['COUNT_POS_D_prev3']
# Delete 4
del testdf['C_prev4'],testdf['D_prev4'],testdf['ATM_C_prev4'],testdf['ATM_D_prev4'],testdf['BRANCH_C_prev4']
del testdf['BRANCH_D_prev4'],testdf['IB_C_prev4'],testdf['IB_D_prev4'],testdf['MB_C_prev4'],testdf['MB_D_prev4']
del testdf['POS_C_prev4'],testdf['POS_D_prev4'],testdf['count_C_prev4'],testdf['count_D_prev4'],testdf['COUNT_ATM_C_prev4']
del testdf['COUNT_ATM_D_prev4'],testdf['COUNT_BRANCH_C_prev4'],testdf['COUNT_BRANCH_D_prev4'],testdf['COUNT_IB_C_prev4']
del testdf['COUNT_IB_D_prev4'],testdf['COUNT_MB_C_prev4'],testdf['COUNT_MB_D_prev4'],testdf['COUNT_POS_C_prev4'],testdf['COUNT_POS_D_prev4']
# Delete 5
del testdf['C_prev5'],testdf['D_prev5'],testdf['ATM_C_prev5'],testdf['ATM_D_prev5'],testdf['BRANCH_C_prev5']
del testdf['BRANCH_D_prev5'],testdf['IB_C_prev5'],testdf['IB_D_prev5'],testdf['MB_C_prev5'],testdf['MB_D_prev5']
del testdf['POS_C_prev5'],testdf['POS_D_prev5'],testdf['count_C_prev5'],testdf['count_D_prev5'],testdf['COUNT_ATM_C_prev5']
del testdf['COUNT_ATM_D_prev5'],testdf['COUNT_BRANCH_C_prev5'],testdf['COUNT_BRANCH_D_prev5'],testdf['COUNT_IB_C_prev5']
del testdf['COUNT_IB_D_prev5'],testdf['COUNT_MB_C_prev5'],testdf['COUNT_MB_D_prev5'],testdf['COUNT_POS_C_prev5'],testdf['COUNT_POS_D_prev5']
# Delete 6
del testdf['C_prev6'],testdf['D_prev6'],testdf['ATM_C_prev6'],testdf['ATM_D_prev6'],testdf['BRANCH_C_prev6']
del testdf['BRANCH_D_prev6'],testdf['IB_C_prev6'],testdf['IB_D_prev6'],testdf['MB_C_prev6'],testdf['MB_D_prev6']
del testdf['POS_C_prev6'],testdf['POS_D_prev6'],testdf['count_C_prev6'],testdf['count_D_prev6'],testdf['COUNT_ATM_C_prev6']
del testdf['COUNT_ATM_D_prev6'],testdf['COUNT_BRANCH_C_prev6'],testdf['COUNT_BRANCH_D_prev6'],testdf['COUNT_IB_C_prev6']
del testdf['COUNT_IB_D_prev6'],testdf['COUNT_MB_C_prev6'],testdf['COUNT_MB_D_prev6'],testdf['COUNT_POS_C_prev6'],testdf['COUNT_POS_D_prev6']

In [53]:
testdf.shape

(200000, 289)

In [54]:
# Create New variables
# 1
testdf['CuIniCRDiff1'] = ((testdf.fillna(0)['custinit_CR_amt_prev1']/testdf.fillna(0)['custinit_CR_cnt_prev1'])-
(testdf.fillna(0)['custinit_CR_amt_prev2']/testdf.fillna(0)['custinit_CR_cnt_prev2']))
testdf['CuIniDRDiff1'] = ((testdf.fillna(0)['custinit_DR_amt_prev1']/testdf.fillna(0)['custinit_DR_cnt_prev1'])-
(testdf.fillna(0)['custinit_DR_amt_prev2']/testdf.fillna(0)['custinit_DR_cnt_prev2']))
testdf['ATMWithDiff1'] = (testdf.fillna(0)['ATM_amt_prev1'] - testdf.fillna(0)['ATM_amt_prev2'])
testdf['ATMAvDwDiff1'] = (((testdf.fillna(0)['ATM_CW_Amt_prev1']/testdf.fillna(0)['ATM_CW_Cnt_prev1']).replace(np.inf, 0))-
((testdf.fillna(0)['ATM_CW_Amt_prev2']/testdf.fillna(0)['ATM_CW_Cnt_prev2']).replace(np.inf, 0)))
testdf['BCWAvDwDiff1'] = (((testdf.fillna(0)['BRN_CW_Amt_prev1']/testdf.fillna(0)['BRN_CW_Cnt_prev1']).replace(np.inf, 0))-
((testdf.fillna(0)['BRN_CW_Amt_prev2']/testdf.fillna(0)['BRN_CW_Cnt_prev2']).replace(np.inf, 0)))
testdf['BCDAvDwDiff1'] = (((testdf.fillna(0)['BRN_CASH_Dep_Amt_prev1']/testdf.fillna(0)['BRN_CASH_Dep_Cnt_prev1']).replace(np.inf, 0))-
((testdf.fillna(0)['BRN_CASH_Dep_Amt_prev2']/testdf.fillna(0)['BRN_CASH_Dep_Cnt_prev2']).replace(np.inf, 0)))
testdf['CNTDiff1'] = (testdf.fillna(0)['CNR_prev1'] - testdf.fillna(0)['CNR_prev2'])
testdf['BALDiff1'] = (testdf.fillna(0)['BAL_prev1'] - testdf.fillna(0)['BAL_prev2'])
testdf['EOPDiff1'] = (testdf.fillna(0)['EOP_prev1'] - testdf.fillna(0)['EOP_prev2'])
testdf['CRAMBDiff1'] = (testdf.fillna(0)['CR_AMB_Prev1'] - testdf.fillna(0)['CR_AMB_Prev2'])
# 2
testdf['CuIniCRDiff2'] = ((testdf.fillna(0)['custinit_CR_amt_prev2']/testdf.fillna(0)['custinit_CR_cnt_prev2'])-
(testdf.fillna(0)['custinit_CR_amt_prev3']/testdf.fillna(0)['custinit_CR_cnt_prev3']))
testdf['CuIniDRDiff2'] = ((testdf.fillna(0)['custinit_DR_amt_prev2']/testdf.fillna(0)['custinit_DR_cnt_prev2'])-
(testdf.fillna(0)['custinit_DR_amt_prev3']/testdf.fillna(0)['custinit_DR_cnt_prev3']))
testdf['ATMWithDiff2'] = (testdf.fillna(0)['ATM_amt_prev2'] - testdf.fillna(0)['ATM_amt_prev3'])
testdf['ATMAvDwDiff2'] = (((testdf.fillna(0)['ATM_CW_Amt_prev2']/testdf.fillna(0)['ATM_CW_Cnt_prev2']).replace(np.inf, 0))-
((testdf.fillna(0)['ATM_CW_Amt_prev3']/testdf.fillna(0)['ATM_CW_Cnt_prev3']).replace(np.inf, 0)))
testdf['BCWAvDwDiff2'] = (((testdf.fillna(0)['BRN_CW_Amt_prev2']/testdf.fillna(0)['BRN_CW_Cnt_prev2']).replace(np.inf, 0))-
((testdf.fillna(0)['BRN_CW_Amt_prev3']/testdf.fillna(0)['BRN_CW_Cnt_prev3']).replace(np.inf, 0)))
testdf['BCDAvDwDiff2'] = (((testdf.fillna(0)['BRN_CASH_Dep_Amt_prev2']/testdf.fillna(0)['BRN_CASH_Dep_Cnt_prev2']).replace(np.inf, 0))-
((testdf.fillna(0)['BRN_CASH_Dep_Amt_prev3']/testdf.fillna(0)['BRN_CASH_Dep_Cnt_prev3']).replace(np.inf, 0)))
testdf['CNTDiff2'] = (testdf.fillna(0)['CNR_prev2'] - testdf.fillna(0)['CNR_prev3'])
testdf['BALDiff2'] = (testdf.fillna(0)['BAL_prev2'] - testdf.fillna(0)['BAL_prev3'])
testdf['EOPDiff2'] = (testdf.fillna(0)['EOP_prev2'] - testdf.fillna(0)['EOP_prev3'])
testdf['CRAMBDiff2'] = (testdf.fillna(0)['CR_AMB_Prev2'] - testdf.fillna(0)['CR_AMB_Prev3'])
# 3
testdf['CuIniCRDiff3'] = ((testdf.fillna(0)['custinit_CR_amt_prev3']/testdf.fillna(0)['custinit_CR_cnt_prev3'])-
(testdf.fillna(0)['custinit_CR_amt_prev4']/testdf.fillna(0)['custinit_CR_cnt_prev4']))
testdf['CuIniDRDiff3'] = ((testdf.fillna(0)['custinit_DR_amt_prev3']/testdf.fillna(0)['custinit_DR_cnt_prev3'])-
(testdf.fillna(0)['custinit_DR_amt_prev4']/testdf.fillna(0)['custinit_DR_cnt_prev4']))
testdf['ATMWithDiff3'] = (testdf.fillna(0)['ATM_amt_prev3'] - testdf.fillna(0)['ATM_amt_prev4'])
testdf['ATMAvDwDiff3'] = (((testdf.fillna(0)['ATM_CW_Amt_prev3']/testdf.fillna(0)['ATM_CW_Cnt_prev3']).replace(np.inf, 0))-
((testdf.fillna(0)['ATM_CW_Amt_prev4']/testdf.fillna(0)['ATM_CW_Cnt_prev4']).replace(np.inf, 0)))
testdf['BCWAvDwDiff3'] = (((testdf.fillna(0)['BRN_CW_Amt_prev3']/testdf.fillna(0)['BRN_CW_Cnt_prev3']).replace(np.inf, 0))-
((testdf.fillna(0)['BRN_CW_Amt_prev4']/testdf.fillna(0)['BRN_CW_Cnt_prev4']).replace(np.inf, 0)))
testdf['BCDAvDwDiff3'] = (((testdf.fillna(0)['BRN_CASH_Dep_Amt_prev3']/testdf.fillna(0)['BRN_CASH_Dep_Cnt_prev3']).replace(np.inf, 0))-
((testdf.fillna(0)['BRN_CASH_Dep_Amt_prev4']/testdf.fillna(0)['BRN_CASH_Dep_Cnt_prev4']).replace(np.inf, 0)))
testdf['CNTDiff3'] = (testdf.fillna(0)['CNR_prev3'] - testdf.fillna(0)['CNR_prev4'])
testdf['BALDiff3'] = (testdf.fillna(0)['BAL_prev3'] - testdf.fillna(0)['BAL_prev4'])
testdf['EOPDiff3'] = (testdf.fillna(0)['EOP_prev3'] - testdf.fillna(0)['EOP_prev4'])
testdf['CRAMBDiff3'] = (testdf.fillna(0)['CR_AMB_Prev3'] - testdf.fillna(0)['CR_AMB_Prev4'])
# 4
testdf['CuIniCRDiff4'] = ((testdf.fillna(0)['custinit_CR_amt_prev4']/testdf.fillna(0)['custinit_CR_cnt_prev4'])-
(testdf.fillna(0)['custinit_CR_amt_prev5']/testdf.fillna(0)['custinit_CR_cnt_prev5']))
testdf['CuIniDRDiff4'] = ((testdf.fillna(0)['custinit_DR_amt_prev4']/testdf.fillna(0)['custinit_DR_cnt_prev4'])-
(testdf.fillna(0)['custinit_DR_amt_prev5']/testdf.fillna(0)['custinit_DR_cnt_prev5']))
testdf['ATMWithDiff4'] = (testdf.fillna(0)['ATM_amt_prev4'] - testdf.fillna(0)['ATM_amt_prev5'])
testdf['ATMAvDwDiff4'] = (((testdf.fillna(0)['ATM_CW_Amt_prev4']/testdf.fillna(0)['ATM_CW_Cnt_prev4']).replace(np.inf, 0))-
((testdf.fillna(0)['ATM_CW_Amt_prev5']/testdf.fillna(0)['ATM_CW_Cnt_prev5']).replace(np.inf, 0)))
testdf['BCWAvDwDiff4'] = (((testdf.fillna(0)['BRN_CW_Amt_prev4']/testdf.fillna(0)['BRN_CW_Cnt_prev4']).replace(np.inf, 0))-
((testdf.fillna(0)['BRN_CW_Amt_prev5']/testdf.fillna(0)['BRN_CW_Cnt_prev5']).replace(np.inf, 0)))
testdf['BCDAvDwDiff4'] = (((testdf.fillna(0)['BRN_CASH_Dep_Amt_prev4']/testdf.fillna(0)['BRN_CASH_Dep_Cnt_prev4']).replace(np.inf, 0))-
((testdf.fillna(0)['BRN_CASH_Dep_Amt_prev5']/testdf.fillna(0)['BRN_CASH_Dep_Cnt_prev5']).replace(np.inf, 0)))
testdf['CNTDiff4'] = (testdf.fillna(0)['CNR_prev4'] - testdf.fillna(0)['CNR_prev5'])
testdf['BALDiff4'] = (testdf.fillna(0)['BAL_prev4'] - testdf.fillna(0)['BAL_prev5'])
testdf['EOPDiff4'] = (testdf.fillna(0)['EOP_prev4'] - testdf.fillna(0)['EOP_prev5'])
testdf['CRAMBDiff4'] = (testdf.fillna(0)['CR_AMB_Prev4'] - testdf.fillna(0)['CR_AMB_Prev5'])
# 5
testdf['CuIniCRDiff5'] = ((testdf.fillna(0)['custinit_CR_amt_prev5']/testdf.fillna(0)['custinit_CR_cnt_prev5'])-
(testdf.fillna(0)['custinit_CR_amt_prev6']/testdf.fillna(0)['custinit_CR_cnt_prev6']))
testdf['CuIniDRDiff5'] = ((testdf.fillna(0)['custinit_DR_amt_prev5']/testdf.fillna(0)['custinit_DR_cnt_prev5'])-
(testdf.fillna(0)['custinit_DR_amt_prev6']/testdf.fillna(0)['custinit_DR_cnt_prev6']))
testdf['ATMWithDiff5'] = (testdf.fillna(0)['ATM_amt_prev5'] - testdf.fillna(0)['ATM_amt_prev6'])
testdf['ATMAvDwDiff5'] = (((testdf.fillna(0)['ATM_CW_Amt_prev5']/testdf.fillna(0)['ATM_CW_Cnt_prev5']).replace(np.inf, 0))-
((testdf.fillna(0)['ATM_CW_Amt_prev6']/testdf.fillna(0)['ATM_CW_Cnt_prev6']).replace(np.inf, 0)))
testdf['BCWAvDwDiff5'] = (((testdf.fillna(0)['BRN_CW_Amt_prev5']/testdf.fillna(0)['BRN_CW_Cnt_prev5']).replace(np.inf, 0))-
((testdf.fillna(0)['BRN_CW_Amt_prev6']/testdf.fillna(0)['BRN_CW_Cnt_prev6']).replace(np.inf, 0)))
testdf['BCDAvDwDiff5'] = (((testdf.fillna(0)['BRN_CASH_Dep_Amt_prev5']/testdf.fillna(0)['BRN_CASH_Dep_Cnt_prev5']).replace(np.inf, 0))-
((testdf.fillna(0)['BRN_CASH_Dep_Amt_prev6']/testdf.fillna(0)['BRN_CASH_Dep_Cnt_prev6']).replace(np.inf, 0)))
testdf['CNTDiff5'] = (testdf.fillna(0)['CNR_prev5'] - testdf.fillna(0)['CNR_prev6'])
testdf['BALDiff5'] = (testdf.fillna(0)['BAL_prev5'] - testdf.fillna(0)['BAL_prev6'])
testdf['EOPDiff5'] = (testdf.fillna(0)['EOP_prev5'] - testdf.fillna(0)['EOP_prev6'])
testdf['CRAMBDiff5'] = (testdf.fillna(0)['CR_AMB_Prev5'] - testdf.fillna(0)['CR_AMB_Prev6'])

In [55]:
# Delete the old columns
# Delete 1
del testdf['custinit_CR_amt_prev1'],testdf['custinit_DR_amt_prev1'],testdf['custinit_CR_cnt_prev1'],testdf['custinit_DR_cnt_prev1']
del testdf['ATM_amt_prev1'],testdf['ATM_CW_Amt_prev1'],testdf['ATM_CW_Cnt_prev1'],testdf['BRN_CW_Amt_prev1']
del testdf['BRN_CW_Cnt_prev1'],testdf['BRN_CASH_Dep_Amt_prev1'],testdf['BRN_CASH_Dep_Cnt_prev1'],testdf['CNR_prev1']
del testdf['BAL_prev1'],testdf['EOP_prev1'],testdf['CR_AMB_Prev1']
# Delete 2
del testdf['custinit_CR_amt_prev2'],testdf['custinit_DR_amt_prev2'],testdf['custinit_CR_cnt_prev2'],testdf['custinit_DR_cnt_prev2']
del testdf['ATM_amt_prev2'],testdf['ATM_CW_Amt_prev2'],testdf['ATM_CW_Cnt_prev2'],testdf['BRN_CW_Amt_prev2']
del testdf['BRN_CW_Cnt_prev2'],testdf['BRN_CASH_Dep_Amt_prev2'],testdf['BRN_CASH_Dep_Cnt_prev2'],testdf['CNR_prev2']
del testdf['BAL_prev2'],testdf['EOP_prev2'],testdf['CR_AMB_Prev2']
# Delete 3
del testdf['custinit_CR_amt_prev3'],testdf['custinit_DR_amt_prev3'],testdf['custinit_CR_cnt_prev3'],testdf['custinit_DR_cnt_prev3']
del testdf['ATM_amt_prev3'],testdf['ATM_CW_Amt_prev3'],testdf['ATM_CW_Cnt_prev3'],testdf['BRN_CW_Amt_prev3']
del testdf['BRN_CW_Cnt_prev3'],testdf['BRN_CASH_Dep_Amt_prev3'],testdf['BRN_CASH_Dep_Cnt_prev3'],testdf['CNR_prev3']
del testdf['BAL_prev3'],testdf['EOP_prev3'],testdf['CR_AMB_Prev3']
# Delete 4
del testdf['custinit_CR_amt_prev4'],testdf['custinit_DR_amt_prev4'],testdf['custinit_CR_cnt_prev4'],testdf['custinit_DR_cnt_prev4']
del testdf['ATM_amt_prev4'],testdf['ATM_CW_Amt_prev4'],testdf['ATM_CW_Cnt_prev4'],testdf['BRN_CW_Amt_prev4']
del testdf['BRN_CW_Cnt_prev4'],testdf['BRN_CASH_Dep_Amt_prev4'],testdf['BRN_CASH_Dep_Cnt_prev4'],testdf['CNR_prev4']
del testdf['BAL_prev4'],testdf['EOP_prev4'],testdf['CR_AMB_Prev4']
# Delete 5
del testdf['custinit_CR_amt_prev5'],testdf['custinit_DR_amt_prev5'],testdf['custinit_CR_cnt_prev5'],testdf['custinit_DR_cnt_prev5']
del testdf['ATM_amt_prev5'],testdf['ATM_CW_Amt_prev5'],testdf['ATM_CW_Cnt_prev5'],testdf['BRN_CW_Amt_prev5']
del testdf['BRN_CW_Cnt_prev5'],testdf['BRN_CASH_Dep_Amt_prev5'],testdf['BRN_CASH_Dep_Cnt_prev5'],testdf['CNR_prev5']
del testdf['BAL_prev5'],testdf['EOP_prev5'],testdf['CR_AMB_Prev5']
# Delete 6
del testdf['custinit_CR_amt_prev6'],testdf['custinit_DR_amt_prev6'],testdf['custinit_CR_cnt_prev6'],testdf['custinit_DR_cnt_prev6']
del testdf['ATM_amt_prev6'],testdf['ATM_CW_Amt_prev6'],testdf['ATM_CW_Cnt_prev6'],testdf['BRN_CW_Amt_prev6']
del testdf['BRN_CW_Cnt_prev6'],testdf['BRN_CASH_Dep_Amt_prev6'],testdf['BRN_CASH_Dep_Cnt_prev6'],testdf['CNR_prev6']
del testdf['BAL_prev6'],testdf['EOP_prev6'],testdf['CR_AMB_Prev6']

In [56]:
testdf.shape

(200000, 249)

In [57]:
# New Loan Features
testdf['PREUNSECLOSLN'] = (testdf.fillna(0)['AGRI_PREM_CLOSED_PREVQ1']+testdf.fillna(0)['CC_PREM_CLOSED_PREVQ1']+
testdf.fillna(0)['EDU_PREM_CLOSED_PREVQ1']+testdf.fillna(0)['PL_PREM_CLOSED_PREVQ1']+
testdf.fillna(0)['BL_PREM_CLOSED_PREVQ1']+testdf.fillna(0)['OTHER_LOANS_PREM_CLOSED_PREVQ1'])
testdf['PRESECLOSLN'] = (testdf.fillna(0)['AL_CNC_PREM_CLOSED_PREVQ1']+testdf.fillna(0)['AL_PREM_CLOSED_PREVQ1']+
testdf.fillna(0)['CE_PREM_CLOSED_PREVQ1']+testdf.fillna(0)['CV_PREM_CLOSED_PREVQ1']+
testdf.fillna(0)['TL_PREM_CLOSED_PREVQ1']+testdf.fillna(0)['TWL_PREM_CLOSED_PREVQ1'])
testdf['PREINVCLOS'] = (testdf.fillna(0)['RD_PREM_CLOSED_PREVQ1']+testdf.fillna(0)['FD_PREM_CLOSED_PREVQ1'])
testdf['UNSECLOSLN'] = (testdf.fillna(0)['AGRI_Closed_PrevQ1']+testdf.fillna(0)['CC_CLOSED_PREVQ1']+
testdf.fillna(0)['EDU_Closed_PrevQ1']+testdf.fillna(0)['OTHER_LOANS_Closed_PrevQ1']+
testdf.fillna(0)['PL_Closed_PrevQ1']+testdf.fillna(0)['BL_Closed_PrevQ1'])
testdf['SECLOSLN'] = (testdf.fillna(0)['AL_CNC_Closed_PrevQ1']+testdf.fillna(0)['AL_Closed_PrevQ1']+
testdf.fillna(0)['CE_Closed_PrevQ1']+testdf.fillna(0)['CV_Closed_PrevQ1']+
testdf.fillna(0)['GL_Closed_PrevQ1']+testdf.fillna(0)['TL_Closed_PrevQ1']+
testdf.fillna(0)['TWL_Closed_PrevQ1'])
testdf['INVCLOS'] = (testdf.fillna(0)['RD_CLOSED_PREVQ1']+testdf.fillna(0)['FD_CLOSED_PREVQ1'])
testdf['INCCLOSLY'] = (testdf.fillna(0)['DEMAT_CLOSED_PREV1YR']+testdf.fillna(0)['SEC_ACC_CLOSED_PREV1YR'])

In [58]:
del testdf['AGRI_PREM_CLOSED_PREVQ1'],testdf['AL_CNC_PREM_CLOSED_PREVQ1'],testdf['AL_PREM_CLOSED_PREVQ1']
del testdf['BL_PREM_CLOSED_PREVQ1'],testdf['CC_PREM_CLOSED_PREVQ1'],testdf['CE_PREM_CLOSED_PREVQ1']
del testdf['CV_PREM_CLOSED_PREVQ1'],testdf['EDU_PREM_CLOSED_PREVQ1'],testdf['OTHER_LOANS_PREM_CLOSED_PREVQ1']
del testdf['PL_PREM_CLOSED_PREVQ1'],testdf['RD_PREM_CLOSED_PREVQ1'],testdf['FD_PREM_CLOSED_PREVQ1']
del testdf['TL_PREM_CLOSED_PREVQ1'],testdf['TWL_PREM_CLOSED_PREVQ1'],testdf['AGRI_Closed_PrevQ1']
del testdf['AL_CNC_Closed_PrevQ1'],testdf['AL_Closed_PrevQ1'],testdf['BL_Closed_PrevQ1'],testdf['CC_CLOSED_PREVQ1']
del testdf['CE_Closed_PrevQ1'],testdf['CV_Closed_PrevQ1'],testdf['EDU_Closed_PrevQ1'],testdf['GL_Closed_PrevQ1']
del testdf['OTHER_LOANS_Closed_PrevQ1'],testdf['PL_Closed_PrevQ1'],testdf['RD_CLOSED_PREVQ1'],testdf['FD_CLOSED_PREVQ1']
del testdf['TL_Closed_PrevQ1'],testdf['TWL_Closed_PrevQ1'],testdf['SEC_ACC_CLOSED_PREV1YR'],testdf['DEMAT_CLOSED_PREV1YR']

In [59]:
testdf.shape

(200000, 225)

In [60]:
# Active Loan and Investments
testdf['LIVSECLN'] = (testdf['AL_CNC_TAG_LIVE']+testdf['AL_TAG_LIVE']+testdf['CE_TAG_LIVE']+
                      testdf['CV_TAG_LIVE']+testdf['GL_TAG_LIVE']+testdf['HL_TAG_LIVE']+
                      testdf['TL_TAG_LIVE']+testdf['TWL_TAG_LIVE']+testdf['lap_tag_live']+
                      testdf['LAS_TAG_LIVE'])
testdf['LIVUSECLN'] = (testdf['AGRI_TAG_LIVE']+testdf['BL_TAG_LIVE']+testdf['CC_TAG_LIVE']+
                      testdf['EDU_TAG_LIVE']+testdf['OTHER_LOANS_TAG_LIVE']+testdf['PL_TAG_LIVE'])
testdf['LIVINVST'] = (testdf['DEMAT_TAG_LIVE']+testdf['SEC_ACC_TAG_LIVE']+testdf['MF_TAG_LIVE']+
                       testdf['INS_TAG_LIVE']+testdf['RD_TAG_LIVE']+testdf['FD_TAG_LIVE'])

In [61]:
del testdf['AGRI_TAG_LIVE'],testdf['AL_CNC_TAG_LIVE'],testdf['AL_TAG_LIVE'],testdf['CV_TAG_LIVE']
del testdf['BL_TAG_LIVE'],testdf['CC_TAG_LIVE'],testdf['CE_TAG_LIVE'],testdf['DEMAT_TAG_LIVE']
del testdf['EDU_TAG_LIVE'],testdf['GL_TAG_LIVE'],testdf['HL_TAG_LIVE'],testdf['SEC_ACC_TAG_LIVE']
del testdf['INS_TAG_LIVE'],testdf['LAS_TAG_LIVE'],testdf['MF_TAG_LIVE'],testdf['OTHER_LOANS_TAG_LIVE']
del testdf['PL_TAG_LIVE'],testdf['RD_TAG_LIVE'],testdf['FD_TAG_LIVE'],testdf['TL_TAG_LIVE']
del testdf['TWL_TAG_LIVE'],testdf['lap_tag_live']

In [62]:
testdf.shape

(200000, 206)

In [63]:
# Fill missing values
testdf['FRX_PrevQ1']=testdf.fillna(0)['FRX_PrevQ1']
testdf['Billpay_Active_PrevQ1']=testdf.fillna(0)['Billpay_Active_PrevQ1']
testdf['Billpay_Reg_ason_Prev1']=testdf.fillna(0)['Billpay_Reg_ason_Prev1']
testdf['NO_OF_FD_BOOK_PrevQ1']=testdf.fillna(0)['NO_OF_FD_BOOK_PrevQ1']
testdf['Billpay_Active_PrevQ1']=testdf.fillna(0)['Billpay_Active_PrevQ1']
testdf['Billpay_Reg_ason_Prev1']=testdf.fillna(0)['Billpay_Reg_ason_Prev1']

In [64]:
# Total Invest value Q1 and Q2
testdf['TotInvValQ1'] = (testdf.fillna(0)['FD_AMOUNT_BOOK_PrevQ1']+testdf.fillna(0)['RD_AMOUNT_BOOK_PrevQ1']+
testdf.fillna(0)['Total_Invest_in_MF_PrevQ1']+testdf.fillna(0)['Dmat_Investing_PrevQ1'])
testdf['TotInvValQ2'] = (testdf.fillna(0)['FD_AMOUNT_BOOK_PrevQ2']+testdf.fillna(0)['RD_AMOUNT_BOOK_PrevQ2']+
testdf.fillna(0)['Total_Invest_in_MF_PrevQ2']+testdf.fillna(0)['Dmat_Investing_PrevQ2'])
testdf['TotInvCntQ1'] = (testdf.fillna(0)['NO_OF_FD_BOOK_PrevQ1']+testdf.fillna(0)['NO_OF_RD_BOOK_PrevQ1']+
testdf.fillna(0)['count_No_of_MF_PrevQ1'])
testdf['TotInvCntQ2'] = (testdf.fillna(0)['NO_OF_FD_BOOK_PrevQ2']+testdf.fillna(0)['NO_OF_RD_BOOK_PrevQ2']+
testdf.fillna(0)['count_No_of_MF_PrevQ2'])

In [65]:
del testdf['FD_AMOUNT_BOOK_PrevQ1'],testdf['RD_AMOUNT_BOOK_PrevQ1'],testdf['Total_Invest_in_MF_PrevQ1'],testdf['Dmat_Investing_PrevQ1']
del testdf['FD_AMOUNT_BOOK_PrevQ2'],testdf['RD_AMOUNT_BOOK_PrevQ2'],testdf['Total_Invest_in_MF_PrevQ2'],testdf['Dmat_Investing_PrevQ2']
del testdf['NO_OF_FD_BOOK_PrevQ1'],testdf['NO_OF_FD_BOOK_PrevQ2'],testdf['NO_OF_RD_BOOK_PrevQ1'],testdf['NO_OF_RD_BOOK_PrevQ2']
del testdf['count_No_of_MF_PrevQ1'],testdf['count_No_of_MF_PrevQ2']

In [66]:
testdf.shape

(200000, 196)

In [67]:
# replace 0 for missing NaN
testdf['NO_OF_COMPLAINTS']=testdf.fillna(0)['NO_OF_COMPLAINTS']

In [68]:
#Req_Logged_PrevQ1	Req_Resolved_PrevQ1	Query_Logged_PrevQ1	Query_Resolved_PrevQ1	Complaint_Logged_PrevQ1	Complaint_Resolved_PrevQ1
#print (traindf.Complaint_Logged_PrevQ1.value_counts(normalize=True,sort=False))
#print ('Missing % is',(traindf.Complaint_Logged_PrevQ1.isnull().sum()/len(traindf)))

In [69]:
# Replace missing value for Qury/Requests
testdf['Req_Resolved_PrevQ1'] = testdf['Req_Resolved_PrevQ1'].fillna(0.0)
testdf['Query_Resolved_PrevQ1'] = testdf['Query_Resolved_PrevQ1'].fillna(0.0)
testdf['Complaint_Resolved_PrevQ1'] = testdf['Complaint_Resolved_PrevQ1'].fillna(0.0)
testdf['Req_Logged_PrevQ1'] = testdf['Req_Logged_PrevQ1'].fillna(0.0)
testdf['Query_Logged_PrevQ1'] = testdf['Query_Logged_PrevQ1'].fillna(0.0)
testdf['Complaint_Logged_PrevQ1'] = testdf['Complaint_Logged_PrevQ1'].fillna(0.0)

In [70]:
#Replace the symbol > with 1.0
testdf['Query_Resolved_PrevQ1']=testdf['Query_Resolved_PrevQ1'].replace('>', 1.0)
testdf['Req_Resolved_PrevQ1']=testdf['Req_Resolved_PrevQ1'].replace('>', 1.0)
testdf['Complaint_Resolved_PrevQ1']=testdf['Complaint_Resolved_PrevQ1'].replace('>', 1.0)

In [71]:
# Conv the Query/Request as String
testdf['Req_Resolved_PrevQ1']=testdf['Req_Resolved_PrevQ1'].astype(float)
testdf['Query_Resolved_PrevQ1']=testdf['Query_Resolved_PrevQ1'].astype(float)
testdf['Complaint_Resolved_PrevQ1']=testdf['Complaint_Resolved_PrevQ1'].astype(float)
testdf['Req_Logged_PrevQ1']=testdf['Req_Resolved_PrevQ1'].astype(float)
testdf['Query_Logged_PrevQ1']=testdf['Query_Resolved_PrevQ1'].astype(float)
testdf['Complaint_Logged_PrevQ1']=testdf['Complaint_Resolved_PrevQ1'].astype(float)

In [72]:
# Create a new variable to gather happiness index
testdf['Reqresperc']=testdf.fillna(0.0)['Req_Resolved_PrevQ1']/testdf.fillna(0.0)['Req_Logged_PrevQ1']
testdf['Qryresperc']=testdf.fillna(0.0)['Query_Resolved_PrevQ1']/testdf.fillna(0.0)['Query_Logged_PrevQ1']
testdf['Cmpresperc']=testdf.fillna(0.0)['Complaint_Resolved_PrevQ1']/testdf.fillna(0.0)['Complaint_Logged_PrevQ1']

In [73]:
# replace 0 for Nan for new variables
#traindf['Reqresperc']=traindf.fillna(0)['Reqresperc']
#traindf['Qryresperc']=traindf.fillna(0)['Qryresperc']
#traindf['Cmpresperc']=traindf.fillna(0)['Cmpresperc']
del testdf['Req_Resolved_PrevQ1'],testdf['Query_Resolved_PrevQ1'],testdf['Complaint_Resolved_PrevQ1'],testdf['Req_Logged_PrevQ1'],testdf['Query_Logged_PrevQ1'],testdf['Complaint_Logged_PrevQ1']
testdf.shape

(200000, 193)

In [74]:
del testdf['AGRI_DATE'],testdf['AL_CNC_DATE'],testdf['AL_DATE'],testdf['BL_DATE'],testdf['CE_DATE']
del testdf['CV_DATE'],testdf['EDU_DATE'],testdf['GL_DATE'],testdf['LAP_DATE'],testdf['LAS_DATE']
del testdf['OTHER_LOANS_DATE'],testdf['PL_DATE'],testdf['TL_DATE'],testdf['TWL_DATE']
testdf.shape

(200000, 179)

In [75]:
testdf['Responders']=0

In [76]:
testdf = testdf[cols_consider]
testdf.shape

In [77]:
# Idnetify the non numric cols for encoding
nonnumcols = testdf.dtypes[testdf.dtypes == "object"].index
print (len(nonnumcols))
print (nonnumcols)

12
Index(['gender_bin', 'RBI_Class_Audit', 'Billpay_Reg_ason_Prev1_N',
       'FRX_PrevQ1_N', 'HNW_CATEGORY', 'EMAIL_UNSUBSCRIBE', 'OCCUP_ALL_NEW',
       'FINAL_WORTH_prev1', 'ENGAGEMENT_TAG_prev1', 'EFT_SELF_TRANSFER_PrevQ1',
       'Charges_cnt_PrevQ1_N', 'Billpay_Active_PrevQ1_N'],
      dtype='object')


In [78]:
testdf = pd.get_dummies(testdf, columns=['gender_bin', 'RBI_Class_Audit', 'Billpay_Reg_ason_Prev1_N',
       'FRX_PrevQ1_N', 'HNW_CATEGORY', 'EMAIL_UNSUBSCRIBE', 'OCCUP_ALL_NEW',
       'FINAL_WORTH_prev1', 'ENGAGEMENT_TAG_prev1', 'EFT_SELF_TRANSFER_PrevQ1',
       'Charges_cnt_PrevQ1_N', 'Billpay_Active_PrevQ1_N'], drop_first=True)
print ('After One hot encoding',testdf.shape)
#'AGEBKT'

After One hot encoding (200000, 97)


In [79]:
print (traindf.shape, testdf.shape)

(300000, 97) (200000, 97)


In [71]:
listA = list(traindf.columns.values)
listB = list(testdf.columns.values)

In [72]:
print (len(listA), len(listB))
"""
for item in listA:
    if item not in listB:
        print (item)"""

200 199


'\nfor item in listA:\n    if item not in listB:\n        print (item)'

In [64]:
#allcols = list(traindf.columns.values)
#print (len(allcols))

195


# Restart from Here

In [2]:
#traindf = pd.read_csv('/Users/Vishy/Files/AVDatahack/Churn/NewData/ntraindf.csv')
traindf = pd.read_csv('/data/ntraindf.csv')
print (traindf.shape)

(300000, 97)


In [3]:
#testdf = pd.read_csv('/Users/Vishy/Files/AVDatahack/Churn/NewData/ntestdf.csv')
testdf = pd.read_csv('/data/ntestdf.csv')
print (testdf.shape)

(200000, 97)


In [4]:
del testdf['Responders']
testdf.shape

(200000, 96)

In [5]:
data = []
for f in traindf.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif traindf[f].dtype == float:
        level = 'interval'
    elif traindf[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    
    # Defining the data type 
    dtype = traindf[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [6]:
meta.head()

,role,level,keep,dtype
varname,,,,
UCIC_ID,input,ordinal,True,int64
Percent_Change_in_Credits,input,interval,True,float64
Recency_of_CR_TXN,input,interval,True,float64
Recency_of_DR_TXN,input,interval,True,float64
Recency_of_BRANCH_TXN,input,interval,True,float64


In [7]:
# Checking for the ratio of missing value
Total = traindf.isnull().sum().sort_values(ascending=False)
percent = ((traindf.isnull().sum()/traindf.isnull().count())*100).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :percent, 'Missing Total':Total})
missing_data.head(10)

,Missing Ratio,Missing Total
Percent_Change_in_Credits,32.138333,96415
Recency_of_CR_TXN,11.064667,33194
Recency_of_DR_TXN,7.750667,23252
Recency_of_BRANCH_TXN,6.122667,18368
Recency_of_Activity,0.159000,477
Billpay_Active_PrevQ1_N_B_1,0.000000,0
CNTDiff4,0.000000,0
I_NRV_PrevQ2,0.000000,0
TotInvCntQ1,0.000000,0
CR_AMB_Drop_Build_2,0.000000,0


In [8]:
# imputing missing values
mean_imp = Imputer(missing_values=np.NaN, strategy='mean', axis=0)
mode_imp = Imputer(missing_values=np.NaN, strategy='most_frequent', axis=0)
traindf['Recency_of_Activity'] = mean_imp.fit_transform(traindf[['Recency_of_Activity']]).ravel()
traindf['Recency_of_BRANCH_TXN'] = mean_imp.fit_transform(traindf[['Recency_of_BRANCH_TXN']]).ravel()
traindf['Recency_of_DR_TXN'] = mean_imp.fit_transform(traindf[['Recency_of_DR_TXN']]).ravel()
traindf['Recency_of_CR_TXN'] = mean_imp.fit_transform(traindf[['Recency_of_CR_TXN']]).ravel()
traindf['Percent_Change_in_Credits'] = mode_imp.fit_transform(traindf[['Percent_Change_in_Credits']]).ravel()

In [9]:
# Checking for the ratio of missing value
Total = traindf.isnull().sum().sort_values(ascending=False)
percent = ((traindf.isnull().sum()/traindf.isnull().count())*100).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :percent, 'Missing Total':Total})
missing_data.head()

,Missing Ratio,Missing Total
Billpay_Active_PrevQ1_N_B_1,0.0,0
ATMWithDiff2,0.0,0
CASH_WD_AMT_Last6,0.0,0
CASH_WD_CNT_Last6,0.0,0
CR_AMB_Drop_Build_1,0.0,0


In [10]:
# Creating Interaction variables
v = meta[(meta.level == 'interval') & (meta.keep)].index
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
interactions = pd.DataFrame(data=poly.fit_transform(traindf[v]), columns=poly.get_feature_names(v))
interactions.drop(v, axis=1, inplace=True)  # Remove the original columns
# Concat the interaction variables to the train data
print('Before creating interactions we have {} variables in train'.format(traindf.shape[1]))
train = pd.concat([traindf, interactions], axis=1)
print('After creating interactions we have {} variables in train'.format(traindf.shape[1]))

Before creating interactions we have 97 variables in train
After creating interactions we have 97 variables in train


# Repeat for Test Data

In [11]:
data = []
for f in testdf.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif testdf[f].dtype == float:
        level = 'interval'
    elif testdf[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    
    # Defining the data type 
    dtype = testdf[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [12]:
meta.head()

,role,level,keep,dtype
varname,,,,
UCIC_ID,input,ordinal,True,int64
Percent_Change_in_Credits,input,interval,True,float64
Recency_of_CR_TXN,input,interval,True,float64
Recency_of_DR_TXN,input,interval,True,float64
Recency_of_BRANCH_TXN,input,interval,True,float64


In [13]:
# Checking for the ratio of missing value
Total = testdf.isnull().sum().sort_values(ascending=False)
percent = ((testdf.isnull().sum()/testdf.isnull().count())*100).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :percent, 'Missing Total':Total})
missing_data.head(10)

,Missing Ratio,Missing Total
Percent_Change_in_Credits,32.0940,64188
Recency_of_CR_TXN,11.0790,22158
Recency_of_DR_TXN,7.7090,15418
Recency_of_BRANCH_TXN,6.1235,12247
Recency_of_Activity,0.1600,320
Billpay_Active_PrevQ1_N_B_1,0.0000,0
CNTDiff4,0.0000,0
I_NRV_PrevQ2,0.0000,0
TotInvCntQ1,0.0000,0
CR_AMB_Drop_Build_2,0.0000,0


In [14]:
# imputing missing values
mean_imp = Imputer(missing_values=np.NaN, strategy='mean', axis=0)
mode_imp = Imputer(missing_values=np.NaN, strategy='most_frequent', axis=0)
testdf['Recency_of_Activity'] = mean_imp.fit_transform(testdf[['Recency_of_Activity']]).ravel()
testdf['Recency_of_BRANCH_TXN'] = mean_imp.fit_transform(testdf[['Recency_of_BRANCH_TXN']]).ravel()
testdf['Recency_of_DR_TXN'] = mean_imp.fit_transform(testdf[['Recency_of_DR_TXN']]).ravel()
testdf['Recency_of_CR_TXN'] = mean_imp.fit_transform(testdf[['Recency_of_CR_TXN']]).ravel()
testdf['Percent_Change_in_Credits'] = mode_imp.fit_transform(testdf[['Percent_Change_in_Credits']]).ravel()

In [15]:
# Checking for the ratio of missing value
Total = testdf.isnull().sum().sort_values(ascending=False)
percent = ((testdf.isnull().sum()/testdf.isnull().count())*100).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :percent, 'Missing Total':Total})
missing_data.head()

,Missing Ratio,Missing Total
Billpay_Active_PrevQ1_N_B_1,0.0,0
Charges_cnt_PrevQ1_N_D_3,0.0,0
CASH_WD_AMT_Last6,0.0,0
CASH_WD_CNT_Last6,0.0,0
CR_AMB_Drop_Build_1,0.0,0


In [16]:
# Creating Interaction variables
v = meta[(meta.level == 'interval') & (meta.keep)].index
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
interactions = pd.DataFrame(data=poly.fit_transform(testdf[v]), columns=poly.get_feature_names(v))
interactions.drop(v, axis=1, inplace=True)  # Remove the original columns
# Concat the interaction variables to the train data
print('Before creating interactions we have {} variables in train'.format(testdf.shape[1]))
train = pd.concat([traindf, interactions], axis=1)
print('After creating interactions we have {} variables in train'.format(testdf.shape[1]))

Before creating interactions we have 96 variables in train
After creating interactions we have 96 variables in train


In [17]:
def xgb_lift(preds, dtrain):
    y = dtrain.get_label()
    y_pred = preds
    df = pd.DataFrame({'proba':y_pred, 'y':y})
    # 0.2 signifies the top 2 decile
    lift_score = df.sort_values('proba', ascending=False).iloc[:int(len(df)*0.2), 1].sum() / df['y'].sum()
    return [('lift', lift_score)]

In [18]:
target_train = traindf['Responders'].values
id_test = testdf['UCIC_ID'].values

train = np.array(traindf.drop(['Responders','UCIC_ID'], axis = 1))
test = np.array(testdf.drop(['UCIC_ID'], axis = 1))

xgb_preds = []

In [19]:
K = 3
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [20]:
start = time()

for train_index, test_index in kf.split(train):
    train_X, valid_X = train[train_index], train[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]
    xgb_params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.5, 'colsample_bytree': 0.9, 
                  'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000,  watchlist, maximize=True, feval=xgb_lift,
                      verbose_eval=100, early_stopping_rounds=50)
                        
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

end = time()
print ('Total time for training is',end-start)
# 9192.85 for k=4
# 2643.70 for k=2 
# 1019.95 for k=2 with lift
# 798.57 for k=2 with lift & new feats
# 1242.73 for k=3 with lift & 82 new features

[0]	train-auc:0.785026	valid-auc:0.783267	train-lift:0.562422	valid-lift:0.563264
Multiple eval metrics have been passed: 'valid-lift' will be used for early stopping.

Will train until valid-lift hasn't improved in 50 rounds.
[100]	train-auc:0.839826	valid-auc:0.838981	train-lift:0.635868	valid-lift:0.633046
[200]	train-auc:0.849556	valid-auc:0.848574	train-lift:0.648703	valid-lift:0.643639
[300]	train-auc:0.856588	valid-auc:0.854617	train-lift:0.654585	valid-lift:0.651263
[400]	train-auc:0.860521	valid-auc:0.857817	train-lift:0.660785	valid-lift:0.654639
[500]	train-auc:0.863347	valid-auc:0.859588	train-lift:0.664204	valid-lift:0.658072
[600]	train-auc:0.865612	valid-auc:0.860924	train-lift:0.667072	valid-lift:0.660342
[700]	train-auc:0.867492	valid-auc:0.861856	train-lift:0.669506	valid-lift:0.661739
Stopping. Best iteration:
[742]	train-auc:0.868097	valid-auc:0.862003	train-lift:0.670781	valid-lift:0.662612

[0]	train-auc:0.78515	valid-auc:0.783525	train-lift:0.567478	valid-lift:0.

In [21]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

out1 = pd.DataFrame({'UCIC_ID': id_test, 'Responders': preds})

In [22]:
out1.to_csv("pred8.csv", index=False)

In [28]:
X_train = traindf.drop(['UCIC_ID', 'Responders'], axis=1)
y_train = traindf['Responders']
id_test = testdf['UCIC_ID'].values
X_test = traindf.drop(['UCIC_ID', 'Responders'], axis=1)
feat_labels = X_train.columns
start = time()
clf = RandomForestClassifier(verbose=3,random_state=7,n_jobs=-1,max_features='sqrt',n_estimators=60, min_samples_split=2)
print ('Initializing for training')
clf.fit(X_train, y_train)
importances = clf.feature_importances_
end = time()
print ("Training complete in: ", end-start, "seconds")
indices = np.argsort(clf.feature_importances_)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]], importances[indices[f]]))

Initializing for training
building tree 1 of 60building tree 2 of 60building tree 3 of 60building tree 4 of 60



building tree 5 of 60
building tree 6 of 60
building tree 7 of 60
building tree 8 of 60
building tree 9 of 60
building tree 10 of 60
building tree 11 of 60
building tree 12 of 60
building tree 13 of 60
building tree 14 of 60
building tree 15 of 60
building tree 16 of 60
building tree 17 of 60
building tree 18 of 60
building tree 19 of 60
building tree 20 of 60
building tree 21 of 60
building tree 22 of 60
building tree 23 of 60
building tree 24 of 60
building tree 25 of 60
building tree 26 of 60
building tree 27 of 60


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   28.7s


building tree 28 of 60
building tree 29 of 60
building tree 30 of 60
building tree 31 of 60
building tree 32 of 60
building tree 33 of 60
building tree 34 of 60
building tree 35 of 60
building tree 36 of 60
building tree 37 of 60
building tree 38 of 60
building tree 39 of 60
building tree 40 of 60
building tree 41 of 60
building tree 42 of 60
building tree 43 of 60
building tree 44 of 60
building tree 45 of 60
building tree 46 of 60
building tree 47 of 60
building tree 48 of 60
building tree 49 of 60
building tree 50 of 60
building tree 51 of 60
building tree 52 of 60
building tree 53 of 60
building tree 54 of 60
building tree 55 of 60
building tree 56 of 60
building tree 57 of 60
building tree 58 of 60
building tree 59 of 60
building tree 60 of 60


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.2min finished


Training complete in:  69.62085294723511 seconds
 1) CR_AMB_Drop_Build_1            0.079967
 2) EOPDiff1                       0.069079
 3) BALDiff1                       0.047737
 4) CRAMBDiff1                     0.039105
 5) CR_AMB_Drop_Build_2            0.030683
 6) CNTDiff1                       0.026712
 7) BALDiff2                       0.024903
 8) EOPDiff2                       0.024693
 9) CRAMBDiff2                     0.022016
10) I_AQB_PrevQ2                   0.020754
11) I_CR_AQB_PrevQ2                0.019465
12) CR_AMB_Drop_Build_3            0.018900
13) vintage                        0.018082
14) I_NRV_PrevQ2                   0.017355
15) BALDiff3                       0.017323
16) CR_AMB_Drop_Build_5            0.017003
17) I_AQB_PrevQ1                   0.016716
18) CR_AMB_Drop_Build_4            0.016658
19) I_CR_AQB_PrevQ1                0.016574
20) I_NRV_PrevQ1                   0.016231
21) I_CNR_PrevQ1                   0.015894
22) Recency_of_DR_TXN      

In [30]:
sfm = SelectFromModel(clf, threshold='median', prefit=True)
print('Number of features before selection: {}'.format(X_train.shape[1]))
n_features = sfm.transform(X_train).shape[1]
print('Number of features after selection: {}'.format(n_features))
selected_vars = list(feat_labels[sfm.get_support()])

Number of features before selection: 95
Number of features after selection: 48


In [46]:
ntraindf = traindf[selected_vars]
ntraindfy = traindf['Responders']
ntestdf = testdf[selected_vars]
print ('The shape of train and test and y is',ntraindf.shape, ntestdf.shape,ntraindfy.shape  )

The shape of train and test and y is (300000, 48) (200000, 48) (300000,)


In [54]:
#clf = RandomForestClassifier(max_features= 'sqrt',n_estimators=300, min_samples_split=2, random_state=7, n_jobs=-1)
#clf = clf.fit(ntraindf,ntraindfy)
preds = clf.predict_proba(ntestdf)[:,1]

In [57]:
out1 = pd.DataFrame({'UCIC_ID': id_test, 'Responders': preds})

In [59]:
out1.to_csv("pred8.csv", index=False)

In [2]:
#traindf = pd.read_csv('/Users/Vishy/Files/AVDatahack/Churn/NewData/ntraindf.csv')
traindf = pd.read_csv('/data/ntraindf.csv')
print (traindf.shape)

#testdf = pd.read_csv('/Users/Vishy/Files/AVDatahack/Churn/NewData/ntestdf.csv')
testdf = pd.read_csv('/data/ntestdf.csv')
print (testdf.shape)

(300000, 97)
(200000, 97)


In [3]:
# imputing missing values
mean_imp = Imputer(missing_values=np.NaN, strategy='mean', axis=0)
mode_imp = Imputer(missing_values=np.NaN, strategy='most_frequent', axis=0)
traindf['Recency_of_Activity'] = mean_imp.fit_transform(traindf[['Recency_of_Activity']]).ravel()
traindf['Recency_of_BRANCH_TXN'] = mean_imp.fit_transform(traindf[['Recency_of_BRANCH_TXN']]).ravel()
traindf['Recency_of_DR_TXN'] = mean_imp.fit_transform(traindf[['Recency_of_DR_TXN']]).ravel()
traindf['Recency_of_CR_TXN'] = mean_imp.fit_transform(traindf[['Recency_of_CR_TXN']]).ravel()
traindf['Percent_Change_in_Credits'] = mode_imp.fit_transform(traindf[['Percent_Change_in_Credits']]).ravel()

In [4]:
# imputing missing values
mean_imp = Imputer(missing_values=np.NaN, strategy='mean', axis=0)
mode_imp = Imputer(missing_values=np.NaN, strategy='most_frequent', axis=0)
testdf['Recency_of_Activity'] = mean_imp.fit_transform(testdf[['Recency_of_Activity']]).ravel()
testdf['Recency_of_BRANCH_TXN'] = mean_imp.fit_transform(testdf[['Recency_of_BRANCH_TXN']]).ravel()
testdf['Recency_of_DR_TXN'] = mean_imp.fit_transform(testdf[['Recency_of_DR_TXN']]).ravel()
testdf['Recency_of_CR_TXN'] = mean_imp.fit_transform(testdf[['Recency_of_CR_TXN']]).ravel()
testdf['Percent_Change_in_Credits'] = mode_imp.fit_transform(testdf[['Percent_Change_in_Credits']]).ravel()

In [5]:
del testdf['Responders']

In [8]:
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

In [10]:
# Preprocessing 
id_test = testdf['UCIC_ID'].values
target_train = traindf['Responders'].values

train = traindf.drop(['Responders','UCIC_ID'], axis = 1)
test = testdf.drop(['UCIC_ID'], axis = 1)

In [11]:
print(train.values.shape, test.values.shape)

(300000, 95) (200000, 95)


In [12]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
#                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [13]:
# RandomForest params
rf_params = {}
rf_params['n_estimators'] = 200
rf_params['max_depth'] = 6
rf_params['min_samples_split'] = 70
rf_params['min_samples_leaf'] = 30


# ExtraTrees params
et_params = {}
et_params['n_estimators'] = 155
et_params['max_features'] = 0.3
et_params['max_depth'] = 6
et_params['min_samples_split'] = 40
et_params['min_samples_leaf'] = 18


# XGBoost params
xgb_params = {}
xgb_params['objective'] = 'binary:logistic'
xgb_params['learning_rate'] = 0.04
xgb_params['n_estimators'] = 490
xgb_params['max_depth'] = 4
xgb_params['subsample'] = 0.9
xgb_params['colsample_bytree'] = 0.9  
xgb_params['min_child_weight'] = 10

"""
# LightGBM params
lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 650
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 500
lgb_params['seed'] = 99


lgb_params2 = {}
lgb_params2['n_estimators'] = 1090
lgb_params2['learning_rate'] = 0.02
lgb_params2['colsample_bytree'] = 0.3   
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['seed'] = 99


lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.02
lgb_params3['seed'] = 99
"""


In [15]:
rf_model = RandomForestClassifier(**rf_params)
et_model = ExtraTreesClassifier(**et_params)
xgb_model = XGBClassifier(**xgb_params)
log_model = LogisticRegression()

In [18]:
from sklearn.model_selection import StratifiedKFold

stack = Ensemble(n_splits=2,
        stacker = log_model,
        base_models = (rf_model, et_model, xgb_model))        
        
y_pred = stack.fit_predict(train, target_train, test) 

Fit RandomForestClassifier fold 1
Fit RandomForestClassifier fold 2
Fit ExtraTreesClassifier fold 1
Fit ExtraTreesClassifier fold 2
Fit XGBClassifier fold 1
Fit XGBClassifier fold 2
Stacker score: 0.85869


In [19]:
sub = pd.DataFrame()
sub['UCIC_ID'] = id_test
sub['Responders'] = y_pred
sub.to_csv('stacked_1.csv', index=False)

In [20]:
sub.head()

,UCIC_ID,Responders
0,337734,0.408974
1,488166,0.051407
2,410785,0.048896
3,389145,0.080743
4,221090,0.200417
